In [1]:
import numpy as np
import os
import pandas as pd
import arcpy

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
gis = arcgis.GIS()
from arcgis.geoanalytics import manage_data

from IPython.display import display, Markdown

In [2]:
dirWork         = os.getcwd()
dirInput        = os.path.join(dirWork, r'input'       )
dirIntermediate = os.path.join(dirWork, r'intermediate')
dirResults      = os.path.join(dirWork, r'results'     )

print("Working Directory: "      + dirWork        )
print("Input Directory: "        + dirInput       )
print("Intermediate Directory: " + dirIntermediate)
print("Results Directory: "      + dirResults     )

Working Directory: E:\GitHub\Census-LEHD-Commute-Patterns
Input Directory: E:\GitHub\Census-LEHD-Commute-Patterns\input
Intermediate Directory: E:\GitHub\Census-LEHD-Commute-Patterns\intermediate
Results Directory: E:\GitHub\Census-LEHD-Commute-Patterns\results


In [3]:
dirIntermediate

'E:\\GitHub\\Census-LEHD-Commute-Patterns\\intermediate'

In [4]:
#input filenames
fname_CityTownship_csv        = os.path.join(dirInput, 'citytownship.csv'                                         )
fname_CityTownship_shp        = os.path.join(dirInput, 'Municipalities_Township' + r'\Municipalities_Township.shp') 
fname_LEHDCommutePatterns_csv = os.path.join(dirInput, 'results_at_blockgroup_wfrc6820212_withcolumnsum.csv'      )
fname_BlockGroups_shp         = os.path.join(dirInput, 'tl_2020_49_bg' + r'\tl_2020_49_bg.shp'                    )
fname_Counties_AGRC_shp       = os.path.join(dirInput, 'Utah_County_Boundaries-shp' + r'\Counties.shp'            )


#intermediate
fname_CityTownshipCounty_BlockGroups_shp = os.path.join(dirIntermediate, 'citytownshipcounty_blockgroup_intersect.shp')
fname_CityTownshipWCodes_shp             = os.path.join(dirIntermediate, 'Municipalities_Townships_withCodes.shp')
fname_CountyWCodes_shp                   = os.path.join(dirIntermediate, 'Counties_withCodes.shp')
fname_CityTownshipWCodesUnionCounty_shp  = os.path.join(dirIntermediate, 'Municipalities_Townships_withCodes_Counties.shp')
fname_CityTownshipCountyWCodes_shp       = os.path.join(dirIntermediate, 'Municipalities_Townships_Counties_withCodes.shp')

#display(dirIntermediate)

#results geodatabase

fname_CityTownshipJSON = os.path.join(dirResults, "citytownshipdata.json")

gdb_LEHDCommutePatterns = "LEHDCommutePatterns.gdb"
fname_LEHDCommutePatterns_gdb = os.path.join(dirResults, gdb_LEHDCommutePatterns)

fname_CityTownshipWCodes = os.path.join(fname_LEHDCommutePatterns_gdb, 'Municipalities_Townships')
#fname_CityTownshipCounty = os.path.join(fname_LEHDCommutePatterns_gdb, 'Municipalities_Townships_Counties')

fname_BlockGroupsWCommutePatterns_Number = os.path.join(fname_LEHDCommutePatterns_gdb, 'BlockGroup_CommutePatterns_Number')
fname_BlockGroupsWCommutePatterns_Percent_SA = os.path.join(fname_LEHDCommutePatterns_gdb, 'BlockGroup_CommutePatterns_Percent_SelectedArea')
fname_BlockGroupsWCommutePatterns_Percent_MU = os.path.join(fname_LEHDCommutePatterns_gdb, 'BlockGroup_CommutePatterns_Percent_MapUnit')

fname_TractsWCommutePatterns_Number = os.path.join(fname_LEHDCommutePatterns_gdb, 'Tract_CommutePatterns_Number')
fname_TractsWCommutePatterns_Percent_SA = os.path.join(fname_LEHDCommutePatterns_gdb, 'Tract_CommutePatterns_Percent_SelectedArea')
fname_TractsWCommutePatterns_Percent_MU = os.path.join(fname_LEHDCommutePatterns_gdb, 'Tract_CommutePatterns_Percent_MapUnit')

fname_CitiesWCommutePatterns_Number = os.path.join(fname_LEHDCommutePatterns_gdb, 'City_CommutePatterns_Number')
fname_CitiesWCommutePatterns_Percent_SA = os.path.join(fname_LEHDCommutePatterns_gdb, 'City_CommutePatterns_Percent_SelectedArea')
fname_CitiesWCommutePatterns_Percent_MU = os.path.join(fname_LEHDCommutePatterns_gdb, 'City_CommutePatterns_Percent_MapUnit')

fname_Counties = os.path.join(fname_LEHDCommutePatterns_gdb, 'Counties')

def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)

#if results geodatabase doesn't exist, create it
print("Checking if " + fname_LEHDCommutePatterns_gdb + " exists...")
if not arcpy.Exists(os.path.join(dirResults, gdb_LEHDCommutePatterns)):
    print("Creating " + fname_LEHDCommutePatterns_gdb + "...")
    arcpy.management.CreateFileGDB(dirResults, gdb_LEHDCommutePatterns)
else:
    print(gdb_LEHDCommutePatterns + " exists...")

#print(AnalysisAreasInput)

Checking if E:\GitHub\Census-LEHD-Commute-Patterns\results\LEHDCommutePatterns.gdb exists...
Creating E:\GitHub\Census-LEHD-Commute-Patterns\results\LEHDCommutePatterns.gdb...


# Create Cities/Townships Feature Class with 3-Digit Code and County FIPS

In [5]:
#read in city/township shapefile, SHORTDESC is common field between this and CODE3 dataset
sdf_CityTownship = pd.DataFrame.spatial.from_featureclass(fname_CityTownship_shp)
display(sdf_CityTownship)

,FID,COUNTYNBR,NAME,COUNTYSEAT,SHORTDESC,UPDATED,FIPS,ENTITYNBR,SALESTAXID,IMSCOLOR,MINNAME,POPLASTCEN,POPLASTEST,GNIS,GlobalID,SHAPE_Leng,SHAPE_Area,SHAPE
0,0,03,Newton,0,NEWTON,2018-09-19,54550,3100.0,047,2,,801,813,1430705,{1C320704-F3E4-4D4F-A542-F6FFE3C36900},0.071275,0.000240,"{""rings"": [[[-112.00138477499996, 41.855487523..."
1,1,12,Eureka,0,EUREKA,2018-09-19,24080,3010.0,009,2,,693,695,1437974,{E1F6AAAD-E49E-4820-A270-D3E7C2131A80},0.100706,0.000395,"{""rings"": [[[-112.09987515199998, 39.957670251..."
2,2,29,Huntsville,0,HUNTSVILLE,2014-10-15,37060,3030.0,019,4,Huntsville,602,638,1428949,{E9A934D8-FE13-4A6D-925E-73A9988E3F17},0.126158,0.000246,"{""rings"": [[[-111.76117535999998, 41.253396941..."
3,3,27,Springdale,0,SPRINGDALE,2014-10-15,71840,3100.0,023,2,Springdale,569,609,1432867,{E6E76C45-D141-47DF-9C96-D3D46F15A2B9},0.193621,0.001208,"{""rings"": [[[-112.97934207899999, 37.197581387..."
4,4,23,Grantsville,0,GRANTSVILLE,2020-06-05,31120,3010.0,023,2,,9152,11568,1428338,{5F5B6417-7EF9-49B5-A689-CBF42953DC9D},0.772545,0.010742,"{""rings"": [[[-112.37537669199997, 40.593511564..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,253,18,COPPERTON METRO TOWNSHIP,0,Copperton,2016-12-27,15720,0.0,,0,Copperton,822,835,,{B3F64B90-565B-44B4-8C7B-87000672B734},0.067294,0.000086,"{""rings"": [[[-112.09934129299995, 40.562296682..."
254,254,18,KEARNS METRO TOWNSHIP,0,Kearns,2016-12-27,40470,0.0,,0,Kearns,35769,36330,,{75A0B23A-775B-44DC-BCCA-4E504BF9E548},0.185692,0.001277,"{""rings"": [[[-111.98660579699998, 40.667567316..."
255,255,18,MAGNA METRO TOWNSHIP,0,Magna,2018-11-14,47290,0.0,,0,Magna,26513,26949,,{28F85E1F-72D3-43E5-BBF4-F85CC28898D5},0.756349,0.010342,"{""rings"": [[[-112.10619664199999, 40.691640141..."
256,256,18,EMIGRATION CANYON METRO TOWNSHIP,0,Emigration Canyon,2016-12-27,22875,0.0,,0,Emigration Canyon,1568,1592,,{29B91C07-4DE5-4395-8337-6E13304256BB},0.350202,0.005034,"{""rings"": [[[-111.74338063899995, 40.818777216..."


In [6]:
#rename METRO TOWNSHIPs to match CODE3 dataset
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'WHITE CITY METRO TOWNSHIP'       , 'SHORTDESC'] = "WHITE CITY METRO TOWNSHIP"
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'EMIGRATION CANYON METRO TOWNSHIP', 'SHORTDESC'] = "EMIGRATION CANYON METRO TOWNSHIP"
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'COPPERTON METRO TOWNSHIP'        , 'SHORTDESC'] = "COPPERTON METRO TOWNSHIP"
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'MAGNA METRO TOWNSHIP'            , 'SHORTDESC'] = "MAGNA METRO TOWNSHIP"
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'KEARNS METRO TOWNSHIP'           , 'SHORTDESC'] = "KEARNS METRO TOWNSHIP"

#give townships a better name
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'WHITE CITY METRO TOWNSHIP'       , 'NAME'] = "White City"
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'EMIGRATION CANYON METRO TOWNSHIP', 'NAME'] = "Emigration Canyon"
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'COPPERTON METRO TOWNSHIP'        , 'NAME'] = "Copperton"
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'MAGNA METRO TOWNSHIP'            , 'NAME'] = "Magna"
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'KEARNS METRO TOWNSHIP'           , 'NAME'] = "Kearns"

In [7]:
#read in township codes data
df_CityTownshipCodes = pd.read_csv(fname_CityTownship_csv)

#consolidate Hideout
df_CityTownshipCodes = df_CityTownshipCodes.loc[df_CityTownshipCodes.SHORTDESC != 'HIDEOUT (WASATCH)']
df_CityTownshipCodes.loc[df_CityTownshipCodes.SHORTDESC == 'HIDEOUT (SUMMIT)', 'SHORTDESC'] = "HIDEOUT"

df_CityTownshipCodes

,SHORTDESC,CODE3
0,AMERICAN FORK,AFK
1,ALTA,ALA
2,ALPINE,ALP
3,BLUFFDALE (SL CO),BDL
4,BLUFFDALE (UTAH CO),BDL
...,...,...
96,WILLARD CITY,WIL
97,WEST JORDAN CITY,WJC
98,WEST POINT,WPT
99,WEST VALLEY CITY,WVC


In [8]:
#join spatial data frame to data frame with codes using SHORTDESC field as key between two
sdf_CityTownshipWCodes = pd.DataFrame.merge(sdf_CityTownship, df_CityTownshipCodes, on='SHORTDESC')

#export to intermediate location so that it can be used in dissolve function later (to remove cities that span counties)
sdf_CityTownshipWCodes.spatial.to_featureclass(fname_CityTownshipWCodes_shp)

display(sdf_CityTownshipWCodes)

,FID,COUNTYNBR,NAME,COUNTYSEAT,SHORTDESC,UPDATED,FIPS,ENTITYNBR,SALESTAXID,IMSCOLOR,MINNAME,POPLASTCEN,POPLASTEST,GNIS,GlobalID,SHAPE_Leng,SHAPE_Area,SHAPE,CODE3
0,2,29,Huntsville,0,HUNTSVILLE,2014-10-15,37060,3030.0,019,4,Huntsville,602,638,1428949,{E9A934D8-FE13-4A6D-925E-73A9988E3F17},0.126158,0.000246,"{""rings"": [[[-111.76117535999998, 41.253396941...",HVL
1,4,23,Grantsville,0,GRANTSVILLE,2020-06-05,31120,3010.0,023,2,,9152,11568,1428338,{5F5B6417-7EF9-49B5-A689-CBF42953DC9D},0.772545,0.010742,"{""rings"": [[[-112.37537669199997, 40.593511564...",GRL
2,5,18,Bluffdale,0,BLUFFDALE (SL CO),2018-11-15,06810,3020.0,019,4,,7693,14699,1425844,{E52CE320-1532-4A57-8DB9-DFA819FBB85B},0.303290,0.002868,"{""rings"": [[[-111.94473146099995, 40.497759794...",BDL
3,7,18,Herriman,0,HERRIMAN TOWN,2019-11-22,34970,3035.0,060,1,,22018,44877,1428675,{3E78F48D-80B9-44DA-AC9F-836A7B4F1B8E},0.617524,0.005943,"{""rings"": [[[-112.06032590299998, 40.481954434...",HER
4,13,18,Taylorsville,0,TAYLORSVILLE CITY,2014-09-15,75360,3105.0,142,4,Taylorsville,59989,60192,1433206,{3ADD19D5-EF23-4257-8048-E355AE283BC6},0.303444,0.002991,"{""rings"": [[[-111.92081851599994, 40.685958404...",TAY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,253,18,Copperton,0,COPPERTON METRO TOWNSHIP,2016-12-27,15720,0.0,,0,Copperton,822,835,,{B3F64B90-565B-44B4-8C7B-87000672B734},0.067294,0.000086,"{""rings"": [[[-112.09934129299995, 40.562296682...",CMT
96,254,18,Kearns,0,KEARNS METRO TOWNSHIP,2016-12-27,40470,0.0,,0,Kearns,35769,36330,,{75A0B23A-775B-44DC-BCCA-4E504BF9E548},0.185692,0.001277,"{""rings"": [[[-111.98660579699998, 40.667567316...",KMT
97,255,18,Magna,0,MAGNA METRO TOWNSHIP,2018-11-14,47290,0.0,,0,Magna,26513,26949,,{28F85E1F-72D3-43E5-BBF4-F85CC28898D5},0.756349,0.010342,"{""rings"": [[[-112.10619664199999, 40.691640141...",MMT
98,256,18,Emigration Canyon,0,EMIGRATION CANYON METRO TOWNSHIP,2016-12-27,22875,0.0,,0,Emigration Canyon,1568,1592,,{29B91C07-4DE5-4395-8337-6E13304256BB},0.350202,0.005034,"{""rings"": [[[-111.74338063899995, 40.818777216...",EMT


In [9]:
#dissolve by code, due to some cities spanning counties and having multiple records in codes and/or munis/townships
deleteIfExists(fname_CityTownshipWCodes)
arcpy.management.Dissolve(fname_CityTownshipWCodes_shp, fname_CityTownshipWCodes, ['CODE3'], [['NAME','FIRST'], ['COUNTYNBR','FIRST']])

sdf_CityTownshipWCodes = pd.DataFrame.spatial.from_featureclass(fname_CityTownshipWCodes)
display(sdf_CityTownshipWCodes)

,OBJECTID,CODE3,FIRST_NAME,FIRST_COUNTYNBR,SHAPE
0,1,AFK,American Fork,25,"{""rings"": [[[-111.77435121873373, 40.353624868..."
1,2,ALA,Alta,18,"{""rings"": [[[-111.59946851855727, 40.577431980..."
2,3,ALP,Alpine,25,"{""rings"": [[[-111.75455302160464, 40.479513910..."
3,4,BDL,Bluffdale,18,"{""rings"": [[[-111.9704071902396, 40.5008379065..."
4,5,BGM,Brigham City,02,"{""rings"": [[[-112.01898409573937, 41.535781784..."
...,...,...,...,...,...
92,93,WIL,Willard,02,"{""rings"": [[[-112.02999503963935, 41.391465965..."
93,94,WJC,West Jordan,18,"{""rings"": [[[-112.00475852158849, 40.638664299..."
94,95,WPT,West Point,06,"{""rings"": [[[-112.10779328273378, 41.143687479..."
95,96,WVC,West Valley City,18,"{""rings"": [[[-111.9626132723846, 40.7260972067..."


In [10]:
df_CityTownship_NameCode = sdf_CityTownshipWCodes[['FIRST_NAME','CODE3','FIRST_COUNTYNBR']].copy()
df_CityTownship_NameCode = df_CityTownship_NameCode.sort_values(by=['FIRST_NAME'])

df_CityTownship_NameCode.columns = ('NAME','CODE3','COUNTYNBR')

display(df_CityTownship_NameCode)

#df_CityTownship_NameCode.to_json(os.path.join(dirResults, "citytownshipdata.json"),orient='records')
#export after adding workers and residents columns from data

,NAME,CODE3,COUNTYNBR
2,Alpine,ALP,25
1,Alta,ALA,18
0,American Fork,AFK,25
3,Bluffdale,BDL,18
5,Bountiful,BNT,06
...,...,...,...
95,West Valley City,WVC,18
90,White City,WHT,18
92,Willard,WIL,02
88,Woodland Hills,WDL,25


In [11]:
#explicit definition of county for cross-county cities
df_CityTownship_NameCode.loc[((df_CityTownship_NameCode['CODE3'] == 'BDL')), 'COUNTYNBR'] = '18'
df_CityTownship_NameCode.loc[((df_CityTownship_NameCode['CODE3'] == 'DRA')), 'COUNTYNBR'] = '18'
df_CityTownship_NameCode.loc[((df_CityTownship_NameCode['CODE3'] == 'HDT')), 'COUNTYNBR'] = '26'
df_CityTownship_NameCode.loc[((df_CityTownship_NameCode['CODE3'] == 'PRK')), 'COUNTYNBR'] = '22'

#read in county dataset
sdf_CountiesAGRC = pd.DataFrame.spatial.from_featureclass(fname_Counties_AGRC_shp)
sdf_CountiesAGRC['FIPS'   ] = sdf_CountiesAGRC['FIPS'].astype(int)
sdf_CountiesAGRC['zCOFIPS'] = 'z' + sdf_CountiesAGRC['FIPS'].astype(str).str.zfill(2) #add 'z' instead of 0, so that columns names won't start with number, and so sorts at end
sdf_CountiesAGRC['CO_NAME'] = sdf_CountiesAGRC['NAME'].str.title() + ' County'
display(sdf_CountiesAGRC)

#join by COUNTYNBR to get CO_FIPS
df_CityTownship_NameCode_wCOFIPS = pd.DataFrame.merge(df_CityTownship_NameCode,sdf_CountiesAGRC,on='COUNTYNBR')
display(df_CityTownship_NameCode_wCOFIPS)


df_CityTownship_NameCode = df_CityTownship_NameCode_wCOFIPS[['NAME_x','CODE3','zCOFIPS']]
df_CityTownship_NameCode.columns = ('NAME','CODE3','zCOFIPS')

display(df_CityTownship_NameCode)

,FID,OBJECTID,COUNTYNBR,ENTITYNBR,ENTITYYR,NAME,FIPS,STATEPLANE,POP_LASTCE,POP_CURRES,GlobalID,FIPS_STR,COLOR4,SHAPE_Leng,SHAPE_Area,SHAPE,zCOFIPS,CO_NAME
0,0,1,03,2.010031e+09,2010.0,CACHE,5,North,112656,128289,{59BEE5FF-00D6-414A-8CD9-3B9CF5FA25FB},49005,2,3.071504,0.328661,"{""rings"": [[[-112.15617197899996, 41.997725941...",z05,Cache County
1,1,2,07,2.010071e+09,2010.0,DUCHESNE,13,Central,18607,19938,{AAE112BD-8D24-46EE-98CD-6689B87A14D1},49013,4,3.966961,0.891751,"{""rings"": [[[-110.25173670299995, 40.832347463...",z13,Duchesne County
2,2,3,06,2.010061e+09,2010.0,DAVIS,11,North,306479,355481,{66DDE457-AF53-4B6D-9A7D-25F29FCCF626},49011,3,2.259091,0.176033,"{""rings"": [[[-111.92092166399999, 41.138676415...",z11,Davis County
3,3,4,14,2.010141e+09,2010.0,MILLARD,27,Central,12503,13188,{DE7E933D-7233-4FBE-8D0C-5E0826EF05D7},49027,3,6.100361,1.843191,"{""rings"": [[[-112.23854898599996, 39.553633436...",z27,Millard County
4,4,5,26,2.010261e+09,2010.0,WASATCH,51,Central,23530,34091,{1BAA04F5-D976-4917-BBF2-BE9C0F868C55},49051,2,3.353038,0.331949,"{""rings"": [[[-111.39572506599995, 40.690187842...",z51,Wasatch County
5,5,6,02,2.010021e+09,2010.0,BOX ELDER,3,North,49975,56046,{2F4FBEBE-32DB-4A2D-A6D1-277679BDF193},49003,4,6.210604,1.880056,"{""rings"": [[[-113.47488592699995, 41.993309448...",z03,Box Elder County
6,6,7,29,2.010291e+09,2010.0,WEBER,57,North,231236,260213,{7F2E8776-C886-4F23-9D40-B861B1FB5391},49057,1,2.973528,0.183659,"{""rings"": [[[-111.92092166399999, 41.138676415...",z57,Weber County
7,7,8,12,2.010121e+09,2010.0,JUAB,23,Central,10246,12017,{7A6D2476-AE18-48E8-820B-CE97B805879F},49023,4,6.802360,0.926379,"{""rings"": [[[-112.17814760399995, 40.011304953...",z23,Juab County
8,8,9,20,2.010201e+09,2010.0,SANPETE,39,Central,27822,30939,{323210BE-8E14-49FE-96FE-1C571E744993},49039,2,3.238312,0.433777,"{""rings"": [[[-111.60347983999998, 39.812889279...",z39,Sanpete County
9,9,10,27,2.010271e+09,2010.0,WASHINGTON,53,South,138115,177556,{A13328EF-DD99-45B8-897C-A9FE2418E7CA},49053,1,3.591173,0.639603,"{""rings"": [[[-113.47458777999998, 37.607168207...",z53,Washington County


,NAME_x,CODE3,COUNTYNBR,FID,OBJECTID,ENTITYNBR,ENTITYYR,NAME_y,FIPS,STATEPLANE,POP_LASTCE,POP_CURRES,GlobalID,FIPS_STR,COLOR4,SHAPE_Leng,SHAPE_Area,SHAPE,zCOFIPS,CO_NAME
0,Alpine,ALP,25,28,29,2.010251e+09,2010.0,UTAH,49,Central,516564,636235,{33E4105B-187C-4C0B-8F1C-D091F44BEFB1},49049,3,5.073804,0.586227,"{'rings': [[[-111.59404929699997, 40.577075300...",z49,Utah County
1,American Fork,AFK,25,28,29,2.010251e+09,2010.0,UTAH,49,Central,516564,636235,{33E4105B-187C-4C0B-8F1C-D091F44BEFB1},49049,3,5.073804,0.586227,"{'rings': [[[-111.59404929699997, 40.577075300...",z49,Utah County
2,Cedar Fort,CDF,25,28,29,2.010251e+09,2010.0,UTAH,49,Central,516564,636235,{33E4105B-187C-4C0B-8F1C-D091F44BEFB1},49049,3,5.073804,0.586227,"{'rings': [[[-111.59404929699997, 40.577075300...",z49,Utah County
3,Cedar Hills,CHL,25,28,29,2.010251e+09,2010.0,UTAH,49,Central,516564,636235,{33E4105B-187C-4C0B-8F1C-D091F44BEFB1},49049,3,5.073804,0.586227,"{'rings': [[[-111.59404929699997, 40.577075300...",z49,Utah County
4,Eagle Mountain,EAG,25,28,29,2.010251e+09,2010.0,UTAH,49,Central,516564,636235,{33E4105B-187C-4C0B-8F1C-D091F44BEFB1},49049,3,5.073804,0.586227,"{'rings': [[[-111.59404929699997, 40.577075300...",z49,Utah County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Washington Terrace,WAT,29,6,7,2.010291e+09,2010.0,WEBER,57,North,231236,260213,{7F2E8776-C886-4F23-9D40-B861B1FB5391},49057,1,2.973528,0.183659,"{'rings': [[[-111.92092166399999, 41.138676415...",z57,Weber County
93,West Haven,WHV,29,6,7,2.010291e+09,2010.0,WEBER,57,North,231236,260213,{7F2E8776-C886-4F23-9D40-B861B1FB5391},49057,1,2.973528,0.183659,"{'rings': [[[-111.92092166399999, 41.138676415...",z57,Weber County
94,Grantsville,GRL,23,17,18,2.010231e+09,2010.0,TOOELE,45,Central,58218,72259,{197FEA12-E2B4-4514-9A3C-963D8244A1E3},49045,2,6.105650,2.003307,"{'rings': [[[-112.44788691599996, 41.017030151...",z45,Tooele County
95,Tooele,TOO,23,17,18,2.010231e+09,2010.0,TOOELE,45,Central,58218,72259,{197FEA12-E2B4-4514-9A3C-963D8244A1E3},49045,2,6.105650,2.003307,"{'rings': [[[-112.44788691599996, 41.017030151...",z45,Tooele County


,NAME,CODE3,zCOFIPS
0,Alpine,ALP,z49
1,American Fork,AFK,z49
2,Cedar Fort,CDF,z49
3,Cedar Hills,CHL,z49
4,Eagle Mountain,EAG,z49
...,...,...,...
92,Washington Terrace,WAT,z57
93,West Haven,WHV,z57
94,Grantsville,GRL,z45
95,Tooele,TOO,z45


# Add Rows for County Level Totals

In [12]:
#read in csv with commute pattern data
df_LEHDCommutePatterns = pd.read_csv(fname_LEHDCommutePatterns_csv)

#drop extra columns for cross-county cities. data in dropped columns is duplicate of non-".1" columns
df_LEHDCommutePatterns = df_LEHDCommutePatterns.drop(columns=['BDL_x.1','DRA_x.1','HDT_x.1','PRK_x.1','BDL_y.1','DRA_y.1','HDT_y.1','PRK_y.1'])

In [13]:
#get all column codes for summing
dfColCodes = df_LEHDCommutePatterns.loc[1:1,'AFK_x':'TTL_y'].T #get all columns from AFK_x through TTL_y
dfColCodes = dfColCodes.reset_index()

#convert to list for ease of calcs in future steps
dColCodes = dfColCodes['index'].values.tolist()
dColCodes

['AFK_x',
 'ALA_x',
 'ALP_x',
 'BDL_x',
 'BGM_x',
 'BNT_x',
 'BRT_x',
 'CDF_x',
 'CEN_x',
 'CHA_x',
 'CHL_x',
 'CLF_x',
 'CLI_x',
 'CMT_x',
 'COA_x',
 'CWH_x',
 'DAN_x',
 'DRA_x',
 'EAG_x',
 'ELK_x',
 'EMT_x',
 'FAR_x',
 'FCS_x',
 'FFD_x',
 'FRR_x',
 'FTH_x',
 'GLA_x',
 'GOS_x',
 'GRL_x',
 'HAR_x',
 'HDT_x',
 'HEB_x',
 'HER_x',
 'HGH_x',
 'HNF_x',
 'HOL_x',
 'HOO_x',
 'HVL_x',
 'IND_x',
 'INT_x',
 'KAY_x',
 'KMS_x',
 'KMT_x',
 'LAY_x',
 'LEH_x',
 'LIN_x',
 'MAP_x',
 'MID_x',
 'MLC_x',
 'MMT_x',
 'MRG_x',
 'MSL_x',
 'MUR_x',
 'MWY_x',
 'NOG_x',
 'NSL_x',
 'OGD_x',
 'OKL_x',
 'ORM_x',
 'PAY_x',
 'PGR_x',
 'PLN_x',
 'PRK_x',
 'PRY_x',
 'PVO_x',
 'PVW_x',
 'ROY_x',
 'RVD_x',
 'RVT_x',
 'SAN_x',
 'SAQ_x',
 'SAR_x',
 'SFK_x',
 'SJC_x',
 'SLC_x',
 'SLM_x',
 'SOG_x',
 'SPV_x',
 'SSL_x',
 'SUN_x',
 'SWE_x',
 'SYR_x',
 'TAY_x',
 'TOO_x',
 'UIN_x',
 'VIN_x',
 'WAT_x',
 'WBG_x',
 'WDL_x',
 'WEB_x',
 'WHT_x',
 'WHV_x',
 'WIL_x',
 'WJC_x',
 'WPT_x',
 'WVC_x',
 'WXC_x',
 'TTL_x',
 'AFK_y',
 'ALA_y',


In [14]:
#normalize (melt) commute data so that columns become rows
df_LEHDCommutePatterns_Normalized = pd.melt(df_LEHDCommutePatterns, id_vars=['fakeid','GEOID'], value_vars=dColCodes)

#populate new fields with data from previous column names (ABC_x/y): CODE3 and Category (_x/_y)
df_LEHDCommutePatterns_Normalized['CODE3'   ] = df_LEHDCommutePatterns_Normalized['variable'].str[0:3]
df_LEHDCommutePatterns_Normalized['CATEGORY'] = df_LEHDCommutePatterns_Normalized['variable'].str[3:5]

#join with City/Township data using CODE3
df_LEHDCommutePatterns_Normalized_wCOFIPS = pd.DataFrame.merge(df_CityTownship_NameCode, df_LEHDCommutePatterns_Normalized, on='CODE3')
display(df_LEHDCommutePatterns_Normalized_wCOFIPS)

#get county totals for each GEOID by aggregating (groupby)
df_LEHDCommutePatterns_CountyTotals_Normalized = df_LEHDCommutePatterns_Normalized_wCOFIPS.groupby(['zCOFIPS','GEOID','CATEGORY'],as_index=False).agg({'value':np.sum})

#create column names for county level totals
df_LEHDCommutePatterns_CountyTotals_Normalized['zCOFIPS_wCat'] = df_LEHDCommutePatterns_CountyTotals_Normalized['zCOFIPS'] + df_LEHDCommutePatterns_CountyTotals_Normalized['CATEGORY']

#create pivot table to reverse normalization, returning structure to original format, but this time with county columns
df_LEHDCommutePatterns_CountyTotals = df_LEHDCommutePatterns_CountyTotals_Normalized.pivot(index='GEOID', columns='zCOFIPS_wCat')['value']

display(df_LEHDCommutePatterns_CountyTotals)

,NAME,CODE3,zCOFIPS,fakeid,GEOID,variable,value,CATEGORY
0,Alpine,ALP,z49,0,colSUM,ALP_x,4470,_x
1,Alpine,ALP,z49,0,490011001001,ALP_x,0,_x
2,Alpine,ALP,z49,1,490011001002,ALP_x,0,_x
3,Alpine,ALP,z49,2,490011001003,ALP_x,0,_x
4,Alpine,ALP,z49,3,490011002001,ALP_x,0,_x
...,...,...,...,...,...,...,...,...
391875,Morgan,MRG,z29,2014,490572112021,MRG_y,0,_y
391876,Morgan,MRG,z29,2015,490572112022,MRG_y,7,_y
391877,Morgan,MRG,z29,2016,490572112023,MRG_y,2,_y
391878,Morgan,MRG,z29,2017,490572112024,MRG_y,0,_y


zCOFIPS_wCat,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
GEOID,,,,,,,,,,,,,,,,,,
490011001001,0,0,5,0,1,1,20,20,0,0,0,0,13,10,0,1,5,3
490011001002,1,0,5,4,0,0,11,53,0,3,0,0,13,20,0,0,3,7
490011001003,1,0,12,11,0,0,59,62,0,1,3,0,19,13,2,1,5,3
490011002001,1,0,0,0,0,0,9,20,0,1,2,1,12,5,0,2,0,1
490011002002,0,0,0,5,0,0,1,27,0,1,0,0,3,24,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490572112022,15,8,202,268,12,7,42,297,1,3,4,0,29,60,2,1,354,592
490572112023,4,4,45,108,2,2,6,127,1,0,2,1,7,19,0,0,83,235
490572112024,5,4,22,81,0,0,4,106,0,1,0,0,3,15,1,0,71,213


In [15]:
#add county columns to original dataset
df_LEHDCommutePatternsWCounty = pd.DataFrame.merge(df_LEHDCommutePatterns, df_LEHDCommutePatterns_CountyTotals, on='GEOID')
df_LEHDCommutePatterns = df_LEHDCommutePatternsWCounty

df_LEHDCommutePatterns

,fakeid,GEOID,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,...,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,0,colSUM,12317,171,4470,6567,9828,20162,76,322,...,6975,13798,22122,9402,256088,235034,11828,7438,108258,100189
1,0,490011001001,0,0,0,0,0,0,0,0,...,0,0,0,0,13,10,0,1,5,3
2,1,490011001002,1,0,0,0,0,1,0,0,...,0,3,0,0,13,20,0,0,3,7
3,2,490011001003,1,0,0,2,1,2,0,0,...,0,1,3,0,19,13,2,1,5,3
4,3,490011002001,1,0,0,0,1,0,0,0,...,0,1,2,1,12,5,0,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2014,490572112021,1,0,1,0,5,3,0,0,...,2,1,3,1,16,10,2,1,197,138
2016,2015,490572112022,2,0,0,0,10,13,0,0,...,1,3,4,0,29,60,2,1,354,592
2017,2016,490572112023,0,0,0,0,3,0,0,0,...,1,0,2,1,7,19,0,0,83,235
2018,2017,490572112024,0,0,0,0,4,0,0,0,...,0,1,0,0,3,15,1,0,71,213


In [16]:
#remove UofU displaced block group
#superceeded by manual reallocation prior to data input prep
#
#display(df_LEHDCommutePatterns[df_LEHDCommutePatterns['GEOID']=='490351108004'])
#
#df_LEHDCommutePatterns = df_LEHDCommutePatterns[df_LEHDCommutePatterns['GEOID'] != '490351108004']

display(df_LEHDCommutePatterns[df_LEHDCommutePatterns['GEOID']=='490351108004'])

,fakeid,GEOID,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,...,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
631,630,490351108004,0,0,2,1,2,5,0,0,...,1,8,11,2,50,53,3,2,14,12


# Quick JSON export with some summary stats

In [17]:
#use row with "colSUM" as GEOID as total
df_LEHDCommutePatterns_ColSum = df_LEHDCommutePatterns[df_LEHDCommutePatterns['GEOID'] == 'colSUM']
df_LEHDCommutePatterns_ColSum

,fakeid,GEOID,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,...,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,0,colSUM,12317,171,4470,6567,9828,20162,76,322,...,6975,13798,22122,9402,256088,235034,11828,7438,108258,100189


In [18]:
#prepare area columns for json
df_people = df_LEHDCommutePatterns_ColSum.loc[:,'AFK_x':'z57_y'].T #transpose
df_people = df_people.reset_index()
df_people['CATEGORY'] = df_people['index'].str[3:5]
df_people['index']    = df_people['index'].str[:3]
df_people

,index,0,CATEGORY
0,AFK,12317,_x
1,ALA,171,_x
2,ALP,4470,_x
3,BDL,6567,_x
4,BGM,9828,_x
...,...,...,...
209,z49,235034,_y
210,z51,11828,_x
211,z51,7438,_y
212,z57,108258,_x


In [19]:
#prepare x columns
df_people_x = df_people[df_people['CATEGORY'] == '_x']
df_people_x.columns = ('CODE3','people_x','CATEGORY')

df_people_x['CODE3'] = df_people_x['CODE3'].str[:3]

df_people_x = df_people_x.drop_duplicates() #some duplicate columns in dataset
display(df_people_x)

#prepare y columns
df_people_y = df_people[df_people['CATEGORY'] == '_y']
df_people_y.columns = ('CODE3','people_y','CATEGORY')

df_people_y['CODE3'] = df_people_y['CODE3'].str[:3]

df_people_y = df_people_y.drop_duplicates() #some duplicate columns in dataset
display(df_people_y)

df_people_x = df_people_x.drop(['CATEGORY'], axis=1)
df_people_y = df_people_y.drop(['CATEGORY'], axis=1)

#combine city/township data with x and y
df_CityTownship_People = pd.DataFrame.merge(df_CityTownship_NameCode,df_people_x, on='CODE3', how='right')
df_CityTownship_People = pd.DataFrame.merge(df_CityTownship_People  ,df_people_y, on='CODE3', how='right')                                           


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,CODE3,people_x,CATEGORY
0,AFK,12317,_x
1,ALA,171,_x
2,ALP,4470,_x
3,BDL,6567,_x
4,BGM,9828,_x
...,...,...,...
204,z43,6975,_x
206,z45,22122,_x
208,z49,256088,_x
210,z51,11828,_x


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,CODE3,people_y,CATEGORY
98,AFK,22429,_y
99,ALA,1475,_y
100,ALP,2785,_y
101,BDL,4403,_y
102,BGM,8977,_y
...,...,...,...
205,z43,13798,_y
207,z45,9402,_y
209,z49,235034,_y
211,z51,7438,_y


In [20]:
#define county dataframe
df_CoNames = sdf_CountiesAGRC[['zCOFIPS','CO_NAME']]

#prepare dataframe for export to JSON, combining city/township
df_CityTownship_Export = pd.DataFrame.merge(df_CityTownship_People, df_CoNames, left_on='CODE3', right_on='zCOFIPS', how='left')

#for any null values, replace with zero(0)
df_CityTownship_Export = df_CityTownship_Export.fillna(0)

#Get city/township name and county name into one field
df_CityTownship_Export.loc[df_CityTownship_Export.CO_NAME != 0, 'NAME'] = df_CityTownship_Export['CO_NAME']

#filter by only 4 columns for export
df_CityTownship_Export = df_CityTownship_Export[['NAME','CODE3','people_x','people_y']]

#change TTL (total) row to Counties subhead for correct positioning in dropdown

#don't include total row
df_CityTownship_Export = df_CityTownship_Export[df_CityTownship_Export['CODE3'] != 'TTL']

#df_CityTownship_Export.loc[((df_CityTownship_Export['CODE3'] == 'TTL')), 'NAME' ] = '---COUNTIES---'
#df_CityTownship_Export.loc[((df_CityTownship_Export['CODE3'] == 'TTL')), 'CODE3'] = 'y000'

#create a first letter field for sorting, to preserve COUNTIES with (z..) at bottom of sort, so can't use name as first sort
#df_CityTownship_Export['firstletter'] = df_CityTownship_Export['CODE3'].str[0:1]
#df_CityTownship_Export = df_CityTownship_Export.sort_values(by=['firstletter','NAME'])
df_CityTownship_Export = df_CityTownship_Export.sort_values(by=['NAME'])

#filter again by only 4 columns for export
df_CityTownship_Export = df_CityTownship_Export[['NAME','CODE3','people_x','people_y']]

#rename columns with 'label' and 'value' to work better in javascript dojo convention
df_CityTownship_Export.columns = ('label','value','people_x','people_y')

#let's see what we got!!
display(df_CityTownship_Export)
    
#export to JSON
df_CityTownship_Export.to_json(fname_CityTownshipJSON,orient='records')

,label,value,people_x,people_y
0,Alpine,ALP,4470,2785
24,Alta,ALA,171,1475
1,American Fork,AFK,12317,22429
25,Bluffdale,BDL,6567,4403
47,Bountiful,BNT,20162,11439
...,...,...,...,...
45,West Valley City,WVC,63926,68427
46,White City,WHT,3030,386
64,Willard,WIL,1017,183
23,Woodland Hills,WDL,834,186


# Update ColCodes with Counties for summing

In [21]:
#get all column codes for summing
dfColCodesWCounty = df_LEHDCommutePatterns.loc[1:1,'AFK_x':'z57_y'].T
dfColCodesWCounty = dfColCodesWCounty.reset_index()
dColCodesWCounty = dfColCodesWCounty['index'].values.tolist()
dColCodesWCounty

['AFK_x',
 'ALA_x',
 'ALP_x',
 'BDL_x',
 'BGM_x',
 'BNT_x',
 'BRT_x',
 'CDF_x',
 'CEN_x',
 'CHA_x',
 'CHL_x',
 'CLF_x',
 'CLI_x',
 'CMT_x',
 'COA_x',
 'CWH_x',
 'DAN_x',
 'DRA_x',
 'EAG_x',
 'ELK_x',
 'EMT_x',
 'FAR_x',
 'FCS_x',
 'FFD_x',
 'FRR_x',
 'FTH_x',
 'GLA_x',
 'GOS_x',
 'GRL_x',
 'HAR_x',
 'HDT_x',
 'HEB_x',
 'HER_x',
 'HGH_x',
 'HNF_x',
 'HOL_x',
 'HOO_x',
 'HVL_x',
 'IND_x',
 'INT_x',
 'KAY_x',
 'KMS_x',
 'KMT_x',
 'LAY_x',
 'LEH_x',
 'LIN_x',
 'MAP_x',
 'MID_x',
 'MLC_x',
 'MMT_x',
 'MRG_x',
 'MSL_x',
 'MUR_x',
 'MWY_x',
 'NOG_x',
 'NSL_x',
 'OGD_x',
 'OKL_x',
 'ORM_x',
 'PAY_x',
 'PGR_x',
 'PLN_x',
 'PRK_x',
 'PRY_x',
 'PVO_x',
 'PVW_x',
 'ROY_x',
 'RVD_x',
 'RVT_x',
 'SAN_x',
 'SAQ_x',
 'SAR_x',
 'SFK_x',
 'SJC_x',
 'SLC_x',
 'SLM_x',
 'SOG_x',
 'SPV_x',
 'SSL_x',
 'SUN_x',
 'SWE_x',
 'SYR_x',
 'TAY_x',
 'TOO_x',
 'UIN_x',
 'VIN_x',
 'WAT_x',
 'WBG_x',
 'WDL_x',
 'WEB_x',
 'WHT_x',
 'WHV_x',
 'WIL_x',
 'WJC_x',
 'WPT_x',
 'WVC_x',
 'WXC_x',
 'TTL_x',
 'AFK_y',
 'ALA_y',


# Add Unincorporated County Areas to Cities/Townships

In [22]:
#filter county dataset with only what we want
sdf_CountiesWCodes = sdf_CountiesAGRC[['zCOFIPS','CO_NAME','SHAPE']]
sdf_CountiesWCodes.columns = ('CODE3','FIRST_NAME','SHAPE')
display(sdf_CountiesWCodes)

#export a couple county spatial dataframes
sdf_CountiesWCodes.spatial.to_featureclass(fname_CountyWCodes_shp) #as shapefile for future use
sdf_CountiesWCodes.spatial.to_featureclass(fname_Counties) #for gdb for use in web app

,CODE3,FIRST_NAME,SHAPE
0,z05,Cache County,"{""rings"": [[[-112.15617197899996, 41.997725941..."
1,z13,Duchesne County,"{""rings"": [[[-110.25173670299995, 40.832347463..."
2,z11,Davis County,"{""rings"": [[[-111.92092166399999, 41.138676415..."
3,z27,Millard County,"{""rings"": [[[-112.23854898599996, 39.553633436..."
4,z51,Wasatch County,"{""rings"": [[[-111.39572506599995, 40.690187842..."
5,z03,Box Elder County,"{""rings"": [[[-113.47488592699995, 41.993309448..."
6,z57,Weber County,"{""rings"": [[[-111.92092166399999, 41.138676415..."
7,z23,Juab County,"{""rings"": [[[-112.17814760399995, 40.011304953..."
8,z39,Sanpete County,"{""rings"": [[[-111.60347983999998, 39.812889279..."
9,z53,Washington County,"{""rings"": [[[-113.47458777999998, 37.607168207..."


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


'E:\\GitHub\\Census-LEHD-Commute-Patterns\\results\\LEHDCommutePatterns.gdb\\Counties'

In [23]:
deleteIfExists(fname_CityTownshipWCodesUnionCounty_shp)

print ("Unioning city/township with county...")

#intersection parcels with analysis area types
arcpy.analysis.Union([fname_CityTownshipWCodes,fname_CountyWCodes_shp], fname_CityTownshipWCodesUnionCounty_shp)

sdf_CityTownshipWCodesCounty = pd.DataFrame.spatial.from_featureclass(fname_CityTownshipWCodesUnionCounty_shp)
sdf_CityTownshipWCodesCounty

Unioning city/township with county...


,FID,FID_Munici,CODE3,FIRST_NAME,FIRST_COUN,Shape_Leng,Shape_Area,FID_Counti,Id,CODE3_1,FIRST_NA_1,SHAPE
0,0,-1,,,,0.000000,0.000000,0,0,z05,Cache County,"{""rings"": [[[-111.50779761699998, 41.999560240..."
1,1,-1,,,,0.000000,0.000000,1,0,z13,Duchesne County,"{""rings"": [[[-109.99900220099995, 40.813577918..."
2,2,-1,,,,0.000000,0.000000,2,0,z11,Davis County,"{""rings"": [[[-112.10299823793908, 41.152736052..."
3,3,-1,,,,0.000000,0.000000,3,0,z27,Millard County,"{""rings"": [[[-112.01601400799998, 39.314566632..."
4,4,-1,,,,0.000000,0.000000,4,0,z51,Wasatch County,"{""rings"": [[[-111.38703771390648, 40.657177632..."
...,...,...,...,...,...,...,...,...,...,...,...,...
153,153,75,SLC,Salt Lake City,18,1.346029,0.030613,27,0,z35,Salt Lake County,"{""rings"": [[[-112.10123724564913, 40.775409823..."
154,154,50,MMT,Magna,18,0.756349,0.010342,27,0,z35,Salt Lake County,"{""rings"": [[[-112.06310238736523, 40.703812554..."
155,155,96,WVC,West Valley City,18,0.614523,0.009884,27,0,z35,Salt Lake County,"{""rings"": [[[-112.06310238736523, 40.703812554..."
156,156,70,SAN,Sandy,18,1.239008,0.006653,27,0,z35,Salt Lake County,"{""rings"": [[[-111.86188875086515, 40.580804996..."


In [24]:
sdf_CityTownshipWCodesCounty.loc[((sdf_CityTownshipWCodesCounty['FID_Munici'] == -1)), 'CODE3'     ] = sdf_CityTownshipWCodesCounty['CODE3_1'   ]
sdf_CityTownshipWCodesCounty.loc[((sdf_CityTownshipWCodesCounty['FID_Munici'] == -1)), 'FIRST_NAME'] = sdf_CityTownshipWCodesCounty['FIRST_NA_1'] + ' Unincorporated'

sdf_CityTownshipWCodesCounty = sdf_CityTownshipWCodesCounty[['FID_Munici','CODE3','FIRST_NAME','SHAPE']]

sdf_CityTownshipWCodesCounty.spatial.to_featureclass(fname_CityTownshipCountyWCodes_shp)
sdf_CityTownshipWCodesCounty

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,FID_Munici,CODE3,FIRST_NAME,SHAPE
0,-1,z05,Cache County Unincorporated,"{""rings"": [[[-111.50779761699998, 41.999560240..."
1,-1,z13,Duchesne County Unincorporated,"{""rings"": [[[-109.99900220099995, 40.813577918..."
2,-1,z11,Davis County Unincorporated,"{""rings"": [[[-112.10299823793908, 41.152736052..."
3,-1,z27,Millard County Unincorporated,"{""rings"": [[[-112.01601400799998, 39.314566632..."
4,-1,z51,Wasatch County Unincorporated,"{""rings"": [[[-111.38703771390648, 40.657177632..."
...,...,...,...,...
153,75,SLC,Salt Lake City,"{""rings"": [[[-112.10123724564913, 40.775409823..."
154,50,MMT,Magna,"{""rings"": [[[-112.06310238736523, 40.703812554..."
155,96,WVC,West Valley City,"{""rings"": [[[-112.06310238736523, 40.703812554..."
156,70,SAN,Sandy,"{""rings"": [[[-111.86188875086515, 40.580804996..."


# Calculate Percent

In [25]:
#read block group shapefile
sdf_BlockGroups = pd.DataFrame.spatial.from_featureclass(fname_BlockGroups_shp)
sdf_LEHDCommutePatterns = pd.DataFrame.merge(sdf_BlockGroups, df_LEHDCommutePatterns, on='GEOID')
display(sdf_LEHDCommutePatterns)

,FID,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,...,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,0,49,035,111306,1,490351113061,Block Group 1,G5030,S,877837.0,...,12,11,14,2,117,70,6,1,15,14
1,1,49,035,111306,2,490351113062,Block Group 2,G5030,S,653262.0,...,0,1,0,3,5,36,0,2,4,6
2,2,49,035,111400,1,490351114001,Block Group 1,G5030,S,381555.0,...,0,9,1,3,6,37,1,0,5,7
3,3,49,035,111400,2,490351114002,Block Group 2,G5030,S,308108.0,...,1,5,3,1,5,16,0,0,2,9
4,4,49,035,111400,3,490351114003,Block Group 3,G5030,S,422651.0,...,0,11,1,0,1,41,0,0,0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,2015,49,057,200900,2,490572009002,Block Group 2,G5030,S,347700.0,...,1,4,2,0,4,29,0,1,122,329
2015,2016,49,057,201200,1,490572012001,Block Group 1,G5030,S,752190.0,...,2,1,1,2,9,28,2,1,264,252
2016,2017,49,057,201200,2,490572012002,Block Group 2,G5030,S,437903.0,...,3,0,12,1,92,27,5,1,595,210
2017,2018,49,057,201301,2,490572013012,Block Group 2,G5030,S,326205.0,...,2,0,0,3,4,20,0,1,76,254


In [26]:
pd.set_option('display.max_columns', None)
df_LEHDCommutePatterns.loc[1:,'AFK_x':]

,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,1,1,1,0,0,0,0,0,1,0,2,1,0,1,0,0,8,0,1,0,1,2,0,0,1,1,2,0,0,0,1,0,0,1,2,0,0,0,0,0,0,0,0,2,0,2,1,4,0,373,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,2,0,2,2,0,0,0,0,1,0,0,0,0,2,0,0,2,0,11,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,246,0,0,5,0,1,1,20,20,0,0,0,0,13,10,0,1,5,3
2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,6,0,0,0,0,6,1,0,1,0,2,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,1,0,268,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,2,0,1,0,0,0,0,1,1,0,0,0,4,0,0,0,2,1,7,0,0,0,2,0,8,1,1,1,0,2,0,0,3,0,22,0,1,0,5,0,0,0,2,0,0,0,0,0,0,0,0,1,0,5,0,7,0,562,1,0,5,4,0,0,11,53,0,3,0,0,13,20,0,0,3,7
3,1,0,0,2,1,2,0,0,0,0,1,0,1,0,0,1,1,3,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,2,0,0,0,0,1,0,2,2,1,0,1,2,5,3,0,0,2,0,0,3,1,0,0,1,0,0,0,0,8,0,2,1,0,5,1,1,1,2,12,1,0,0,2,1,0,1,3,3,0,0,1,0,0,0,0,0,0,5,0,7,0,913,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,2,0,0,0,1,0,0,0,0,0,1,1,0,0,2,0,0,0,0,1,0,0,1,0,1,0,3,6,0,0,0,11,0,0,2,2,0,3,0,2,0,1,0,6,0,0,1,2,2,0,0,0,0,20,0,0,0,4,0,0,1,1,0,0,0,0,0,0,0,0,0,0,3,0,5,1,531,1,0,12,11,0,0,59,62,0,1,3,0,19,13,2,1,5,3
4,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,3,0,1,1,0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,349,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,7,0,0,1,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,6,0,217,1,0,0,0,0,0,9,20,0,1,2,1,12,5,0,2,0,1
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,2,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,2,1,2,1,0,0,0,1,0,1,0,2,0,2,5,0,0,0,0,7,0,0,0,0,6,1,0,0,1,8,0,0,1,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,290,0,0,0,5,0,0,1,27,0,1,0,0,3,24,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [27]:
sdf_LEHDCommutePatterns

,FID,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,BGAcres,SHAPE,fakeid,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,0,49,035,111306,1,490351113061,Block Group 1,G5030,S,877837.0,0.0,+40.6094422,-111.8259365,216.918868,"{'rings': [[[-111.833821, 40.615462], [-111.83...",668,10,1,2,10,0,39,0,0,10,0,0,9,3,0,0,100,1,58,7,2,1,6,0,0,1,0,0,0,3,0,0,3,19,3,1,50,1,0,1,0,24,1,19,14,25,3,1,58,91,4,2,0,58,0,2,17,5,2,18,3,9,0,8,1,13,0,2,0,28,164,0,14,4,57,620,0,0,2,28,1,2,6,35,11,0,0,3,1,1,3,4,1,0,59,0,36,6,1922,5,5,0,1,0,1,2,0,1,0,0,2,1,0,0,62,0,30,0,0,0,2,0,0,0,0,0,0,0,0,0,1,5,0,0,32,0,0,0,0,3,0,4,11,23,3,0,35,39,2,0,0,71,0,0,5,9,0,12,1,6,0,11,0,12,0,1,1,6,76,0,3,2,24,285,0,1,2,33,0,0,1,17,2,0,1,0,0,0,1,0,2,0,29,0,65,1,989,1,0,140,29,2,0,1500,823,12,11,14,2,117,70,6,1,15,14
1,1,49,035,111306,2,490351113062,Block Group 2,G5030,S,653262.0,0.0,+40.6068923,-111.8144187,161.424738,"{'rings': [[[-111.821587, 40.607759], [-111.82...",669,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,9,0,2,2,1,0,1,0,0,0,0,0,0,0,0,0,0,4,0,0,3,1,0,0,0,0,0,0,1,0,0,1,1,4,0,0,0,3,0,1,0,2,0,1,0,0,0,0,0,0,0,0,0,2,7,0,0,0,2,11,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,1,0,74,3,5,0,0,0,6,0,0,1,0,0,1,0,0,0,31,0,17,0,0,0,1,0,0,0,0,0,0,2,0,0,1,2,0,0,7,0,0,0,0,0,0,2,4,14,3,0,24,10,0,0,0,36,1,0,3,6,0,3,0,1,0,1,0,11,0,0,0,2,42,0,0,0,9,153,0,0,0,15,0,0,0,12,1,0,1,0,0,0,0,0,0,0,19,0,32,2,506,0,0,5,18,0,0,55,418,0,1,0,3,5,36,0,2,4,6
2,2,49,035,111400,1,490351114001,Block Group 1,G5030,S,381555.0,0.0,+40.7219003,-111.8796932,94.284721,"{'rings': [[[-111.882588, 40.718416999999995],...",670,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,2,0,4,0,0,1,0,0,0,0,0,0,0,0,0,0,1,3,0,0,5,0,0,0,0,1,0,4,0,3,0,0,1,3,2,0,0,2,0,1,1,3,0,2,0,0,0,0,0,0,1,0,0,2,6,0,0,0,2,16,0,0,0,14,0,0,2,3,1,0,0,0,0,0,0,0,0,0,9,1,11,1,119,5,0,0,0,0,4,0,0,1,0,0,3,0,0,0,9,0,7,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,3,2,7,3,0,9,31,1,0,0,25,0,0,8,3,0,4,0,2,0,9,0,9,0,2,1,4,30,0,1,3,14,239,0,1,1,42,0,0,0,13,3,0,0,0,0,0,0,0,0,0,14,0,31,4,569,1,0,8,23,0,0,90,474,0,9,1,3,6,37,1,0,5,7
3,3,49,035,111400,2,490351114002,Block Group 2,G5030,S,308108.0,0.0,+40.7155063,-111.8797286,76.134594,"{'rings': [[[-111.882623, 40.712824999999995],...",671,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,4,1,0,0,0,2,3,2,0,1,3,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,7,0,0,0,1,19,0,0,1,5,0,0,0,3,3,0,0,0,0,0,0,0,0,0,3,0,4,0,79,1,2,0,2,0,6,0,0,0,0,0,0,0,0,0,11,0,7,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,4,0,3,3,0,6,14,4,0,0,23,0,0,5,7,0,4,0,1,0,5,0,3,0,1,0,1,17,0,0,0,9,126,0,0,1,31,0,0,0,11,1,0,0,0,0,0,0,0,0,0,10,0,22,2,356,1,0,5,13,0,0,58,303,1,5,3,1,5,16,0,0,2,9
4,4,49,035,111400,3,490351114003,Block Group 3,G5030,S,422651.0,0.0,+40.7087113,-111.8797335,104.438922,"{'rin

In [28]:
df_LEHDCommutePatterns.iloc[0,2:216]

AFK_x     12317
ALA_x       171
ALP_x      4470
BDL_x      6567
BGM_x      9828
          ...  
z49_y    235034
z51_x     11828
z51_y      7438
z57_x    108258
z57_y    100189
Name: 0, Length: 214, dtype: object

In [29]:
#initialize Percent_SA distribution dataframe
df_LEHDCommutePatterns_Percent_SA = df_LEHDCommutePatterns

#divide columns AFK_x and after by first row (totals) and columns 2:214
df_LEHDCommutePatterns_Percent_SA.loc[1:,'AFK_x':] = df_LEHDCommutePatterns.loc[1:,'AFK_x':].div(df_LEHDCommutePatterns.iloc[0,2:216])
df_LEHDCommutePatterns_Percent_SA.loc[1:,'AFK_x':] = df_LEHDCommutePatterns.loc[1:,'AFK_x':] * 100
df_LEHDCommutePatterns_Percent_SA

,fakeid,GEOID,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,0,colSUM,12317.000000,171.0,4470.000000,6567.000000,9828.000000,20162.000000,76.0,322.0,7451.000000,254.0,4433.000000,13327.000000,10063.000000,466.0,767.000000,17271.00000,547.000000,22356.000000,14549.000000,1556.000000,817.0,9368.000000,445.0,79.0,3045.000000,2201.000000,805.000000,484.0,5182.000000,3039.000000,339.0,8078.000000,17147.000000,7786.000000,358.00000,15044.000000,3674.000000,279.000000,120.0,50.0,12925.000000,957.000000,18932.000000,31902.000000,29054.000000,5073.000000,4599.000000,17186.000000,31447.00000,14612.000000,1862.000000,926.000000,25805.000000,2272.000000,9109.000000,8408.000000,39719.000000,573.0,44459.000000,9744.000000,15931.000000,3055.000000,3875.000000,2636.000000,39645.000000,4107.000000,18746.000000,4224.000000,23845.000000,48817.000000,5600.000000,13797.000000,19768.000000,32765.000000,94390.000000,4107.000000,7868.000000,16586.000000,12031.000000,2616.000000,3087.000000,12204.000000,31114.000000,16940.000000,621.000000,90.0,4516.000000,168.000000,834.000000,3172.000000,3030.0,5330.000000,1017.000000,59157.000000,4737.000000,63926.000000,5155.000000,1.411279e+06,22429.000000,1475.000000,2785.000000,4403.000000,8977.000000,11439.000000,60.0,52.0,5640.000000,189.000000,845.0,16589.000000,2518.000000,76.0,607.0,20362.000000,183.0,37678.000000,2057.000000,310.0,135.0,9149.000000,60.0,101.0,3082.000000,446.000000,162.0,57.0,1686.000000,1616.000000,7.0,5824.00000,4452.000000,3028.000000,46.0,10117.000000,674.000000,74.000000,13.0,0.0,8034.000000,1153.00000,4298.000000,26964.000000,33202.000000,12517.000000,1921.000000,18321.000000,27496.000000,3019.000000,1254.000000,1821.000000,44412.000000,1168.000000,2792.000000,14796.000000,58127.000000,252.000000,47620.0000,6246.000000,10685.000000,845.000000,11680.000000,906.0,58416.000000,2013.000000,7599.000000,7759.000000,10200.000000,47939.000000,1068.0,3498.000000,12722.000000,29894.000000,259406.000000,1602.000000,6794.000000,12550.000000,34040.000000,1194.000000,636.000000,3763.000000,20760.000000,7716.00000,260.000000,975.000000,3351.000000,54.0,186.0,2169.000000,386.0,3382.000000,183.0,35776.000000,1049.000000,68427.000000,4753.000000,1.411326e+06,13481.000000,10066.000000,146778.000000,109139.000000,1862.000000,1254.000000,556972.000000,683132.000000,6975.000000,13798.000000,22122.000000,9402.000000,256088.000000,235034.000000,11828.000000,7438.000000,108258.000000,100189.000000
1,0,490011001001,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.015007,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.005832,0.000000,0.00000,0.000000,0.000000,0.

In [30]:
sdf_BlockGroups = pd.DataFrame.spatial.from_featureclass(fname_BlockGroups_shp)
sdf_LEHDCommutePatterns_Percent_SA = pd.DataFrame.merge(sdf_BlockGroups, df_LEHDCommutePatterns_Percent_SA, on='GEOID')
sdf_LEHDCommutePatterns_Percent_SA

,FID,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,BGAcres,SHAPE,fakeid,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,0,49,035,111306,1,490351113061,Block Group 1,G5030,S,877837.0,0.0,+40.6094422,-111.8259365,216.918868,"{'rings': [[[-111.833821, 40.615462], [-111.83...",668,0.081189,0.584795,0.044743,0.152277,0.000000,0.193433,0.0,0.0,0.134210,0.000000,0.000000,0.067532,0.029812,0.0,0.000000,0.579005,0.182815,0.259438,0.048113,0.128535,0.122399,0.064048,0.000000,0.0,0.032841,0.000000,0.0,0.0,0.057893,0.000000,0.000000,0.037138,0.110807,0.038531,0.27933,0.332358,0.027218,0.000000,0.833333,0.0,0.185687,0.104493,0.100359,0.043884,0.086047,0.059137,0.021744,0.337484,0.289376,0.027375,0.107411,0.000000,0.224763,0.0,0.021956,0.202188,0.012588,0.34904,0.040487,0.030788,0.056494,0.000000,0.206452,0.037936,0.032791,0.000000,0.010669,0.000000,0.117425,0.335949,0.000000,0.101471,0.020235,0.173966,0.656849,0.000000,0.000000,0.012058,0.232732,0.038226,0.064788,0.049164,0.112490,0.064935,0.000000,0.0,0.066430,0.595238,0.119904,0.094578,0.132013,0.018762,0.000000,0.099735,0.000000,0.056315,0.116392,0.136189,0.022293,0.338983,0.0,0.022712,0.000000,0.008742,3.333333,0.0,0.017730,0.0,0.000000,0.012056,0.039714,0.0,0.0,0.304489,0.0,0.079622,0.000000,0.000000,0.000000,0.021860,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.017170,0.112309,0.000000,0.0,0.316299,0.000000,0.000000,0.0,NaN,0.037341,0.00000,0.093067,0.040795,0.069273,0.023967,0.0,0.191038,0.141839,0.066247,0.000000,0.000000,0.159867,0.000000,0.000000,0.033793,0.015483,0.000000,0.025199,0.01601,0.056153,0.000000,0.094178,0.000000,0.020542,0.000000,0.013160,0.012888,0.058824,0.158535,0.000000,0.085763,0.015721,0.080284,0.109866,0.0,0.014719,0.015936,0.096945,0.000000,0.000000,0.026575,0.081888,0.02592,0.000000,0.102564,0.000000,0.0,0.0,0.046104,0.000000,0.059137,0.000000,0.081060,0.000000,0.094992,0.021039,0.070076,0.007418,0.000000,0.095382,0.026572,0.107411,0.000000,0.269313,0.120475,0.172043,0.079722,0.063285,0.021272,0.045687,0.029783,0.050727,0.013444,0.013856,0.013974
1,1,49,035,111306,2,490351113062,Block Group 2,G5030,S,653262.0,0.0,+40.6068923,-111.8144187,161.424738,"{'rings': [[[-111.821587, 40.607759], [-111.82...",669,0.000000,0.000000,0.000000,0.000000,0.000000,0.004960,0.0,0.0,0.026842,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.052110,0.000000,0.008946,0.013747,0.064267,0.000000,0.010675,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.023328,0.000000,0.00000,0.019942,0.027218,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.003135,0.000000,0.000000,0.021744,0.005819,0.012720,0.000000,0.000000,0.000000,0.011626,0.0,0.010978,0.000000,0.005035,0.00000,0.002249,

In [31]:
sdf_LEHDCommutePatterns_Percent_SA.sum()

FID                                                   2037947
STATEFP     4949494949494949494949494949494949494949494949...
COUNTYFP    0350350350350350350350570350350350350570570570...
TRACTCE     1113061113061114001114001114001114001114002002...
BLKGRPCE    1212356234221212321212234212111131323122431221...
                                  ...                        
z49_y                                                     100
z51_x                                                     100
z51_y                                                     100
z57_x                                                     100
z57_y                                                     100
Length: 229, dtype: object

In [32]:
##create map centered on Salt Lake
##map_areas = gis.map('Salt Lake')
#
##define sdf layer
##sdf_LEHDCommutePatterns_Percent_SA.spatial.plot(map_widget = map_areas,
#                            renderer_type='s',
#                            cmap = 'RdPu',  # use a red to purple color map
#                            symbol_type='simple',
#                            outline_style='s',
#                            outline_color=[0,0,0,255],
#                            line_width=1.0)
#
##display map
#map_areas

In [33]:
#export to GDB
deleteIfExists(fname_BlockGroupsWCommutePatterns_Number)
sdf_LEHDCommutePatterns.spatial.to_featureclass(fname_BlockGroupsWCommutePatterns_Number)


'E:\\GitHub\\Census-LEHD-Commute-Patterns\\results\\LEHDCommutePatterns.gdb\\BlockGroup_CommutePatterns_Number'

In [34]:
deleteIfExists(fname_BlockGroupsWCommutePatterns_Percent_SA)
sdf_LEHDCommutePatterns_Percent_SA.spatial.to_featureclass(fname_BlockGroupsWCommutePatterns_Percent_SA)

'E:\\GitHub\\Census-LEHD-Commute-Patterns\\results\\LEHDCommutePatterns.gdb\\BlockGroup_CommutePatterns_Percent_SelectedArea'

# Group by Tract

In [35]:
#input filenames
fname_Tracts_shp = os.path.join(dirInput, 'Utah_Census_Tracts_2020' + r'\CensusTracts2020.shp')

In [36]:
#read tract shapefile
sdf_Tracts = pd.DataFrame.spatial.from_featureclass(fname_Tracts_shp)
sdf_Tracts

,FID,OBJECTID,STATEFP20,COUNTYFP20,TRACTCE20,GEOID20,NAME20,NAMELSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,SHAPE_Leng,SHAPE_Area,SHAPE
0,0,1,49,049,000901,49049000901,9.01,Census Tract,G5020,S,1.686016e+06,0.0,+40.3051285,-111.6922978,7129.054778,2.902253e+06,"{""rings"": [[[-12434548.4857, 4911361.749799997..."
1,1,2,49,049,002500,49049002500,25,Census Tract,G5020,S,1.034541e+06,0.0,+40.2296149,-111.6497278,5588.845931,1.776891e+06,"{""rings"": [[[-12429589.868, 4898990.177699998]..."
2,2,3,49,049,003401,49049003401,34.01,Census Tract,G5020,S,9.085765e+06,23788.0,+40.0560227,-111.7122570,18878.825275,1.556674e+07,"{""rings"": [[[-12439397.559799999, 4872289.2984..."
3,3,4,49,049,010210,49049010210,102.10,Census Tract,G5020,S,1.581048e+07,0.0,+40.4005884,-111.7306856,26101.425584,2.729233e+07,"{""rings"": [[[-12442557.703499999, 4926512.4487..."
4,4,5,49,049,000505,49049000505,5.05,Census Tract,G5020,S,3.919712e+06,0.0,+40.3700595,-111.7070846,12562.893900,6.760193e+06,"{""rings"": [[[-12437368.7665, 4920654.352499999..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,711,712,49,035,111701,49035111701,1117.01,Census Tract,G5020,S,1.358256e+06,0.0,+40.6934670,-111.8826082,6277.757800,2.365082e+06,"{""rings"": [[[-12455347.0952, 4966754.840000004..."
712,712,713,49,035,111903,49035111903,1119.03,Census Tract,G5020,S,2.072253e+06,0.0,+40.6806830,-111.8627470,7592.371898,3.606962e+06,"{""rings"": [[[-12453471.3611, 4964642.738300003..."
713,713,714,49,035,111905,49035111905,1119.05,Census Tract,G5020,S,1.167718e+06,0.0,+40.6765749,-111.8744366,8615.733516,2.032283e+06,"{""rings"": [[[-12454528.1172, 4964343.751500003..."
714,714,715,49,035,113309,49035113309,1133.09,Census Tract,G5020,S,1.867612e+06,0.0,+40.6881369,-111.9693980,8501.284969,3.251486e+06,"{""rings"": [[[-12465233.158599999, 4967696.3889..."


In [37]:
#calculate tract stats
df_LEHDCommutePatterns_tract = sdf_LEHDCommutePatterns.groupby(['STATEFP','COUNTYFP','TRACTCE'])[dColCodesWCounty].apply(lambda x : x.astype(int).sum())
df_LEHDCommutePatterns_tract = df_LEHDCommutePatterns_tract.reset_index()
display(df_LEHDCommutePatterns_tract)

#caclulate tract Percent_SA stats
df_LEHDCommutePatterns_Percent_SA_tract = sdf_LEHDCommutePatterns_Percent_SA.groupby(['STATEFP','COUNTYFP','TRACTCE'])[dColCodesWCounty].apply(lambda x : x.astype(float).sum())
df_LEHDCommutePatterns_Percent_SA_tract = df_LEHDCommutePatterns_Percent_SA_tract.reset_index()
display(df_LEHDCommutePatterns_Percent_SA_tract)


#df_test = df_LEHDCommutePatterns / df_LEHDCommutePatterns_ColSum

,STATEFP,COUNTYFP,TRACTCE,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,49,001,100100,2,0,0,2,1,3,0,0,0,0,1,2,1,0,0,1,1,3,1,0,0,2,0,0,0,0,0,0,0,0,0,1,2,1,0,2,0,0,0,0,1,0,3,6,1,0,2,3,7,4,1,0,2,0,1,3,3,0,3,2,1,1,0,0,22,0,3,1,1,13,2,1,3,3,16,1,0,1,3,1,0,2,6,3,0,0,1,0,1,0,0,3,1,7,1,12,0,1554,3,1,1,0,0,1,0,0,1,0,0,2,0,0,0,1,0,1,0,0,0,2,0,0,0,1,0,0,0,0,0,2,4,0,0,3,0,0,0,0,3,0,1,1,1,1,0,4,7,0,1,1,15,0,0,2,6,1,12,2,2,0,3,0,15,1,1,2,2,6,0,0,5,0,53,0,1,1,11,0,0,1,3,0,0,0,0,0,0,0,0,1,0,9,0,14,1,1339,2,0,22,15,1,1,90,135,0,4,3,0,45,43,2,2,13,13
1,49,001,100200,1,0,0,0,2,0,0,0,0,0,1,0,0,0,0,2,0,2,1,0,0,0,0,0,0,0,0,0,1,0,0,0,3,0,0,0,0,0,0,0,0,0,1,0,3,0,0,2,2,1,0,0,1,0,0,0,0,1,2,0,0,0,0,1,2,0,0,0,0,1,1,1,3,0,2,1,0,2,1,0,0,0,3,2,0,0,0,0,0,0,0,0,0,3,0,0,0,1050,4,1,3,1,0,1,0,0,0,1,0,1,0,0,0,2,0,4,0,0,0,3,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,4,3,1,2,5,0,0,0,6,0,1,0,3,0,4,9,0,0,3,0,21,0,2,1,0,8,1,0,0,2,38,0,0,2,6,0,0,0,1,2,0,0,0,0,0,1,0,0,0,3,0,16,1,953,3,0,0,8,0,0,24,95,1,4,3,3,18,53,0,2,0,8
2,49,003,960100,0,0,0,0,56,4,0,0,4,0,0,7,4,0,0,1,0,3,1,0,0,2,0,0,6,0,0,0,1,4,0,2,3,1,0,0,6,0,0,0,3,0,4,12,2,1,0,0,0,5,2,1,0,0,18,1,43,0,1,2,1,5,0,9,0,11,8,7,1,1,0,3,0,3,14,1,3,1,1,1,2,7,4,5,0,0,3,0,0,1,1,5,2,3,3,6,1,926,5,3,0,2,144,4,0,0,5,0,0,31,4,0,0,10,0,16,0,0,0,2,0,0,7,0,0,0,0,2,0,4,1,3,0,3,0,0,0,0,8,0,2,29,3,0,0,12,10,0,2,4,17,0,5,4,96,1,18,0,1,2,1,7,14,2,8,4,2,27,1,1,10,6,76,0,7,3,7,0,1,2,4,3,0,1,3,0,0,0,0,4,1,12,0,38,1,1365,67,152,52,91,2,2,50,248,0,2,6,3,14,60,2,4,120,144
3,49,003,960200,1,0,0,0,185,2,0,0,1,0,0,11,6,0,0,0,0,3,0,0,0,1,0,0,5,1,1,0,4,0,0,3,2,1,0,0,1,0,0,0,5,0,1,16,2,0,2,0,3,0,2,1,6,0,16,1,57,0,2,4,0,5,1,13,2,7,24,7,0,1,0,0,3,1,14,0,5,2,1,2,0,8,3,2,0,0,8,0,1,0,0,4,6,5,3,4,0,1528,14,0,1,5,418,11,0,0,7,1,0,49,11,0,0,4,3,36,3,1,0,6,0,0,22,0,0,0,2,8,0,4,3,1,0,2,2,0,0,0,9,0,3,58,17,5,0,16,21,4,5,16,36,0,8,23,228,0,23,5,8,1,5,32,27,16,21,19,6,34,0,3,1,16,203,1,15,9,23,0,4,4,16,10,1,2,6,0,0,2,1,9,4,24,1,53,7,3222,204,454,57,192,2,5,44,506,1,5,6,12,21,121,3,8,140,372
4,49,003,960301,0,0,0,0,79,3,0,0,0,0,0,4,1,0,0,3,0,2,1,0,0,0,0,0,4,1,0,0,4,3,0,0,0,1,0,2,1,1,0,0,4,0,2,7,0,0,0,2,1,1,2,1,3,0,5,4,15,0,9,0,1,2,3,21,6,5,2,3,2,2,1,0,3,2,1,0,1,2,0,0,1,2,1,2,0,0,4,0,0,1,0,2,9,7,1,1,0,1212,12,0,2,2,236,4,0,0,2,0,0,47,4,0,0,4,0,16,2,0,2,6,0,0,7,0,0,0,0,4,0,6,2,6,0,1,0,0,0,0,4,0,1,28,4,4,0,3,10,2,0,5,19,0,4,8,145,0,13,2,2,3,1,20,13,4,13,13,9,20,0,3,2,8,98,0,6,4,20,1,0,1,13,4,0,0,4,0,0,0,0,8,3,12,5,54,3,2039,109,259,29,113,2,0,32,296,3,1,6,4,24,69,0,6,49,216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

,STATEFP,COUNTYFP,TRACTCE,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,49,001,100100,0.016238,0.0,0.000000,0.030455,0.010175,0.014879,0.0,0.0,0.000000,0.000000,0.022558,0.015007,0.009937,0.0,0.000000,0.005790,0.182815,0.013419,0.006873,0.000000,0.000000,0.021349,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.012379,0.011664,0.012844,0.000000,0.013294,0.000000,0.000000,0.0,0.0,0.007737,0.000000,0.015846,0.018808,0.003442,0.000000,0.043488,0.017456,0.022260,0.027375,0.053706,0.000000,0.007750,0.000000,0.010978,0.035680,0.007553,0.00000,0.006748,0.020525,0.006277,0.032733,0.000000,0.000000,0.055492,0.000000,0.016003,0.023674,0.004194,0.026630,0.035714,0.007248,0.015176,0.009156,0.016951,0.024349,0.000000,0.006029,0.024936,0.038226,0.000000,0.016388,0.019284,0.017710,0.000000,0.0,0.022143,0.000000,0.119904,0.000000,0.000000,0.056285,0.098328,0.011833,0.021110,0.018772,0.000000,0.110113,0.013376,0.067797,0.035907,0.000000,0.000000,0.008742,0.0,0.0,0.017730,0.000000,0.000000,0.012056,0.000000,0.0,0.000000,0.004911,0.000000,0.002654,0.000000,0.000000,0.000000,0.021860,0.0,0.0,0.000000,0.224215,0.0,0.0,0.000000,0.000000,0.0,0.034341,0.089847,0.000000,0.0,0.029653,0.000000,0.000000,0.0,0.0,0.037341,0.000000,0.023267,0.003709,0.003012,0.007989,0.000000,0.021833,0.025458,0.000000,0.079745,0.054915,0.033775,0.000000,0.000000,0.013517,0.010322,0.396825,0.025199,0.032020,0.018718,0.000000,0.025685,0.000000,0.025678,0.049677,0.013160,0.025777,0.019608,0.012516,0.000000,0.000000,0.039302,0.000000,0.020431,0.000000,0.014719,0.007968,0.032315,0.000000,0.000000,0.026575,0.014451,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.029568,0.000000,0.025157,0.000000,0.020460,0.021039,0.094875,0.014836,0.000000,0.014989,0.013744,0.053706,0.079745,0.016159,0.019762,0.000000,0.028990,0.013561,0.000000,0.017572,0.018295,0.016909,0.026889,0.012008,0.012975
1,49,001,100200,0.008119,0.0,0.000000,0.000000,0.020350,0.000000,0.0,0.0,0.000000,0.000000,0.022558,0.000000,0.000000,0.0,0.000000,0.011580,0.000000,0.008946,0.006873,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.019298,0.000000,0.0,0.000000,0.017496,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.005282,0.000000,0.010326,0.000000,0.000000,0.011637,0.006360,0.006844,0.000000,0.000000,0.003875,0.000000,0.000000,0.000000,0.000000,0.17452,0.004499,0.000000,0.000000,0.000000,0.000000,0.037936,0.005045,0.000000,0.000000,0.000000,0.000000,0.002048,0.017857,0.007248,0.015176,0.000000,0.002119,0.024349,0.000000,0.012058,0.008312,0.000000,0.000000,0.000000,0.009642,0.011806,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005071,0.000000,0.000000,0.000000,0.074401,0.017834,0.067797

In [38]:
df_LEHDCommutePatterns_Percent_SA_tract.sum()

STATEFP       inf
COUNTYFP      inf
TRACTCE       inf
AFK_x       100.0
ALA_x       100.0
            ...  
z49_y       100.0
z51_x       100.0
z51_y       100.0
z57_x       100.0
z57_y       100.0
Length: 217, dtype: float64

In [39]:
sdf_LEHDCommutePatterns_tract = pd.DataFrame.merge(sdf_Tracts, df_LEHDCommutePatterns_tract, left_on=['STATEFP20','COUNTYFP20','TRACTCE20'], right_on=['STATEFP','COUNTYFP',"TRACTCE"])
display(sdf_LEHDCommutePatterns_tract)

sdf_LEHDCommutePatterns_Percent_SA_tract = pd.DataFrame.merge(sdf_Tracts, df_LEHDCommutePatterns_Percent_SA_tract, left_on=['STATEFP20','COUNTYFP20','TRACTCE20'], right_on=['STATEFP','COUNTYFP',"TRACTCE"])
display(sdf_LEHDCommutePatterns_Percent_SA_tract)

,FID,OBJECTID,STATEFP20,COUNTYFP20,TRACTCE20,GEOID20,NAME20,NAMELSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,SHAPE_Leng,SHAPE_Area,SHAPE,STATEFP,COUNTYFP,TRACTCE,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,0,1,49,049,000901,49049000901,9.01,Census Tract,G5020,S,1.686016e+06,0.0,+40.3051285,-111.6922978,7129.054778,2.902253e+06,"{'rings': [[[-12434548.4857, 4911361.749799997...",49,049,000901,65,0,21,11,14,16,0,1,2,3,18,9,6,2,1,10,0,26,43,4,0,2,0,1,2,3,2,0,15,2,1,14,18,24,0,2,0,0,0,0,8,0,28,20,76,31,16,10,19,33,1,0,25,7,4,5,23,0,537,35,107,1,3,2,305,3,11,3,19,36,23,40,84,36,93,9,7,74,9,2,1,7,34,61,0,1,3,0,5,2,1,3,0,65,1,129,4,2864,147,1,8,9,3,6,0,0,3,0,8,6,0,0,0,22,1,76,12,3,0,4,0,0,0,0,0,0,0,0,0,9,8,15,0,11,0,0,0,0,3,0,1,16,172,123,6,15,28,0,0,1,48,4,1,8,20,0,602,9,78,0,5,0,412,0,4,2,12,59,2,14,43,43,152,5,3,70,31,0,0,3,12,3,0,10,1,0,0,0,0,3,0,23,0,45,6,2543,16,3,88,55,1,0,606,596,4,5,76,3,1522,1739,25,14,62,35
1,1,2,49,049,002500,49049002500,25,Census Tract,G5020,S,1.034541e+06,0.0,+40.2296149,-111.6497278,5588.845931,1.776891e+06,"{'rings': [[[-12429589.868, 4898990.177699998]...",49,049,002500,30,0,2,3,0,1,0,0,0,0,5,0,1,0,0,1,1,6,16,9,0,1,0,0,0,0,2,1,1,0,0,5,3,15,0,2,1,0,1,0,1,1,7,4,44,12,26,2,4,0,1,0,6,1,1,2,4,0,118,46,29,0,1,0,222,0,1,0,6,2,24,8,115,6,7,14,0,87,1,0,0,1,4,4,0,1,1,0,5,0,0,0,0,10,0,5,0,1025,63,0,5,3,4,5,0,0,0,0,3,0,0,0,0,2,2,17,4,2,0,7,0,2,1,0,0,0,0,0,0,3,3,6,0,1,0,0,0,0,2,1,0,10,97,51,6,9,7,0,0,1,13,1,0,6,12,0,275,14,57,0,1,0,540,1,3,1,3,18,1,5,62,10,85,2,1,72,13,1,0,2,4,5,0,0,0,0,1,1,0,1,0,4,1,25,2,1631,0,4,11,37,1,0,75,217,2,2,5,5,831,1268,8,6,8,21
2,2,3,49,049,003401,49049003401,34.01,Census Tract,G5020,S,9.085765e+06,23788.0,+40.0560227,-111.7122570,18878.825275,1.556674e+07,"{'rings': [[[-12439397.559799999, 4872289.2984...",49,049,003401,9,0,1,0,2,3,0,0,1,0,0,2,0,0,1,0,2,2,15,30,0,1,0,0,0,0,17,11,0,0,0,2,2,0,0,0,1,0,0,0,1,0,0,3,16,3,20,3,0,0,1,0,2,0,1,0,8,0,26,257,6,0,1,1,60,0,3,1,2,6,97,7,176,6,4,65,2,84,0,0,0,3,5,6,0,0,0,1,9,0,0,0,0,8,0,4,2,1174,52,0,1,3,1,7,0,2,3,0,0,5,0,0,1,5,1,23,3,1,0,3,0,0,1,0,7,1,1,0,0,11,4,4,0,1,0,0,0,0,2,0,2,14,69,43,15,11,24,3,0,1,25,1,0,7,12,0,164,285,21,0,5,1,232,0,2,2,7,33,15,6,175,15,134,31,4,133,23,0,1,4,16,5,0,8,0,0,0,0,0,0,0,28,1,48,3,1923,3,2,16,50,1,0,44,405,2,6,6,6,909,1268,5,13,16,22
3,3,4,49,049,010210,49049010210,102.10,Census Tract,G5020,S,1.581048e+07,0.0,+40.4005884,-111.7306856,26101.425584,2.729233e+07,"{'rings': [[[-12442557.703499999, 4926512.4487...",49,049,010210,26,0,9,1,1,0,0,0,1,0,64,2,2,0,0,1,0,6,10,0,0,1,0,0,0,0,1,1,0,0,0,1,0,21,0,3,0,0,0,0,0,0,1,2,31,3,2,2,8,0,0,0,7,0,1,1,5,0,35,2,68,1,0,0,25,0,1,0,2,17,0,6,11,5,11,2,1,7,2,0,0,0,7,1,0,0,0,0,0,0,2,0,0,7,0,6,0,451,239,1,35,10,2,7,0,0,5,1,72,9,1,0,1,41,2,90,8,1,

,FID,OBJECTID,STATEFP20,COUNTYFP20,TRACTCE20,GEOID20,NAME20,NAMELSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,SHAPE_Leng,SHAPE_Area,SHAPE,STATEFP,COUNTYFP,TRACTCE,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,0,1,49,049,000901,49049000901,9.01,Census Tract,G5020,S,1.686016e+06,0.0,+40.3051285,-111.6922978,7129.054778,2.902253e+06,"{'rings': [[[-12434548.4857, 4911361.749799997...",49,049,000901,0.527726,0.000000,0.469799,0.167504,0.142450,0.079357,0.000000,0.310559,0.026842,1.181102,0.406046,0.067532,0.059624,0.429185,0.130378,0.057901,0.000000,0.116300,0.295553,0.257069,0.000000,0.021349,0.000000,1.265823,0.065681,0.136302,0.248447,0.000000,0.289464,0.065811,0.294985,0.173310,0.104975,0.308246,0.00000,0.013294,0.000000,0.0,0.000000,0.0,0.061896,0.000000,0.147898,0.062692,0.261582,0.611078,0.347902,0.058187,0.060419,0.225842,0.053706,0.000000,0.096880,0.308099,0.043913,0.059467,0.057907,0.00000,1.207854,0.359195,0.671646,0.032733,0.077419,0.075873,0.769328,0.073046,0.058679,0.071023,0.079681,0.073745,0.410714,0.289918,0.424929,0.109873,0.098527,0.219138,0.088968,0.446159,0.074807,0.076453,0.032394,0.057358,0.109276,0.360094,0.000000,1.111111,0.066430,0.000000,0.599520,0.063052,0.033003,0.056285,0.000000,0.109877,0.021110,0.201796,0.077595,0.202936,0.655401,0.067797,0.287253,0.204406,0.033419,0.052452,0.000000,0.000000,0.053191,0.000000,0.946746,0.036169,0.000000,0.0,0.000000,0.108044,0.546448,0.201709,0.583374,0.967742,0.0,0.043721,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.154533,0.179695,0.495376,0.0,0.108728,0.000000,0.0,0.0,0.0,0.037341,0.000000,0.023267,0.059338,0.518041,0.982664,0.312337,0.081873,0.101833,0.000000,0.000000,0.054915,0.108079,0.342466,0.035817,0.054069,0.034407,0.000000,1.264175,0.144092,0.729995,0.000000,0.042808,0.000000,0.705286,0.000000,0.052639,0.025777,0.117647,0.123073,0.187266,0.400229,0.337997,0.143842,0.058595,0.312110,0.044157,0.557769,0.091069,0.000000,0.000000,0.079724,0.057803,0.038880,0.0,1.025641,0.029842,0.000000,0.000000,0.000000,0.000000,0.088705,0.0,0.064289,0.000000,0.065764,0.126236,0.180185,0.118686,0.029803,0.059954,0.050394,0.053706,0.000000,0.108803,0.087245,0.057348,0.036237,0.343549,0.031908,0.594327,0.739893,0.211363,0.188223,0.057271,0.034934
1,1,2,49,049,002500,49049002500,25,Census Tract,G5020,S,1.034541e+06,0.0,+40.2296149,-111.6497278,5588.845931,1.776891e+06,"{'rings': [[[-12429589.868, 4898990.177699998]...",49,049,002500,0.243566,0.000000,0.044743,0.045683,0.000000,0.004960,0.000000,0.000000,0.000000,0.000000,0.112790,0.000000,0.009937,0.000000,0.000000,0.005790,0.182815,0.026838,0.109973,0.578406,0.000000,0.010675,0.000000,0.000000,0.000000,0.000000,0.248447,0.206612,0.019298,0.000000,0.000000,0.0618

In [40]:
sdf_LEHDCommutePatterns_tract.sum()


FID                                                      255970
OBJECTID                                                 256686
STATEFP20     4949494949494949494949494949494949494949494949...
COUNTYFP20    0490490490490490490490510130490490490490490490...
TRACTCE20     0009010025000034010102100005050005070102099601...
                                    ...                        
z49_y                                                    235034
z51_x                                                     11828
z51_y                                                      7438
z57_x                                                    108258
z57_y                                                    100189
Length: 233, dtype: object

In [41]:
sdf_LEHDCommutePatterns_Percent_SA_tract.sum()

FID                                                      255970
OBJECTID                                                 256686
STATEFP20     4949494949494949494949494949494949494949494949...
COUNTYFP20    0490490490490490490490510130490490490490490490...
TRACTCE20     0009010025000034010102100005050005070102099601...
                                    ...                        
z49_y                                                       100
z51_x                                                       100
z51_y                                                       100
z57_x                                                       100
z57_y                                                       100
Length: 233, dtype: object

In [42]:
sdf_LEHDCommutePatterns_tract.spatial.to_featureclass(fname_TractsWCommutePatterns_Number)

'E:\\GitHub\\Census-LEHD-Commute-Patterns\\results\\LEHDCommutePatterns.gdb\\Tract_CommutePatterns_Number'

In [43]:
sdf_LEHDCommutePatterns_Percent_SA_tract.spatial.to_featureclass(fname_TractsWCommutePatterns_Percent_SA)

'E:\\GitHub\\Census-LEHD-Commute-Patterns\\results\\LEHDCommutePatterns.gdb\\Tract_CommutePatterns_Percent_SelectedArea'

# Group Rows by City

In [45]:

#verify that shape exists and sdf was created correctly and get spatial reference
#print(sdf_CityTownship.iloc[0].SHAPE.spatialReference)
#print(sdf_BlockGroups.iloc[0].SHAPE.spatialReference)
#
##change projection to 26912 to match TAZ Centroids
#sdf_CityTownship.spatial.project(26912)
#sdf_BlockGroups.spatial.project(26912)
#
#print(sdf_CityTownship.iloc[0].SHAPE.spatialReference)
#print(sdf_BlockGroups.iloc[0].SHAPE.spatialReference)

print ("Calculating blockgroup acreage...")

# Execute DeleteField
arcpy.DeleteField_management(fname_BlockGroups_shp, ['BGAcres'])

arcpy.AddField_management(fname_BlockGroups_shp, "BGAcres" , "DOUBLE")
arcpy.CalculateField_management(fname_BlockGroups_shp,
                                "BGAcres",
                                "!shape.area@acres!",
                                "PYTHON_9.3")
print ("Done calculating blockgroup acreage.")


print ("Intersecting blockgroup with city/township/county...")
#intersection parcels with analysis area types
deleteIfExists(fname_CityTownshipCounty_BlockGroups_shp)
arcpy.analysis.Intersect([fname_CityTownshipCountyWCodes_shp,fname_BlockGroups_shp], fname_CityTownshipCounty_BlockGroups_shp)


print ("Calculating piece area...")
arcpy.AddField_management(fname_CityTownshipCounty_BlockGroups_shp, "PieceAcres" , "DOUBLE")
arcpy.CalculateField_management(fname_CityTownshipCounty_BlockGroups_shp,
                                "PieceAcres",
                                "!shape.area@acres!",
                                "PYTHON_9.3")
print ("Calculating Percent_SA of blockgroup...")
arcpy.AddField_management(fname_CityTownshipCounty_BlockGroups_shp, "PortionBG" , "DOUBLE")
arcpy.CalculateField_management(fname_CityTownshipCounty_BlockGroups_shp,
                                "PortionBG",
                                "!PieceAcres!/!BGAcres!",
                                "PYTHON_9.3")


print ("Done")

Calculating blockgroup acreage...
Done calculating blockgroup acreage.
Intersecting blockgroup with city/township/county...
Calculating piece area...
Calculating Percent_SA of blockgroup...
Done


In [46]:
#read newly created shapefile
sdf_CityTownshipCounty_BlockGroups = pd.DataFrame.spatial.from_featureclass(fname_CityTownshipCounty_BlockGroups_shp)


In [47]:
sdf_LEHDCommutePatterns_joinCity = pd.DataFrame.merge(sdf_CityTownshipCounty_BlockGroups,sdf_LEHDCommutePatterns,on=['GEOID','STATEFP','COUNTYFP','TRACTCE','BLKGRPCE','NAMELSAD','MTFCC', 'FUNCSTAT','ALAND','AWATER','INTPTLAT','INTPTLON','BGAcres'])
sdf_LEHDCommutePatterns_joinCity

,FID_x,FID_Munici,Id,FID_Muni_1,CODE3,FIRST_NAME,FID_tl_202,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,BGAcres,PieceAcres,PortionBG,SHAPE_x,FID_y,SHAPE_y,fakeid,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,0,0,0,-1,z05,Cache County Unincorporated,95,49,057,210101,1,490572101011,Block Group 1,G5030,S,86898873.0,19742.0,+41.3700825,-111.9334158,21478.059645,2.821747,0.000131,"{'rings': [[[-111.88525694249688, 41.426031954...",95,"{'rings': [[[-111.975799, 41.384468999999996],...",1918,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,2,2,0,0,0,0,0,7,0,0,0,0,0,0,0,0,2,0,2,0,18,0,0,0,0,2,0,0,1,3,2,1,1,0,0,0,0,2,0,0,2,0,0,0,1,1,1,0,0,0,0,0,0,0,0,2,0,1,0,2,1,84,3,0,0,0,6,2,0,0,2,0,0,7,0,0,0,1,0,5,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,2,1,0,0,4,0,2,8,1,0,0,1,2,0,1,4,3,0,12,2,68,0,5,0,0,0,0,1,1,8,6,6,0,4,0,0,1,4,42,0,9,0,5,0,0,0,3,1,0,0,8,0,0,0,0,2,0,2,0,6,1,347,0,7,17,27,0,1,11,80,0,0,0,1,1,11,0,1,37,128
1,238,5,0,-1,z03,Box Elder County Unincorporated,95,49,057,210101,1,490572101011,Block Group 1,G5030,S,86898873.0,19742.0,+41.3700825,-111.9334158,21478.059645,0.887017,0.000041,"{'rings': [[[-111.95909781387314, 41.433683592...",95,"{'rings': [[[-111.975799, 41.384468999999996],...",1918,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,2,2,0,0,0,0,0,7,0,0,0,0,0,0,0,0,2,0,2,0,18,0,0,0,0,2,0,0,1,3,2,1,1,0,0,0,0,2,0,0,2,0,0,0,1,1,1,0,0,0,0,0,0,0,0,2,0,1,0,2,1,84,3,0,0,0,6,2,0,0,2,0,0,7,0,0,0,1,0,5,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,2,1,0,0,4,0,2,8,1,0,0,1,2,0,1,4,3,0,12,2,68,0,5,0,0,0,0,1,1,8,6,6,0,4,0,0,1,4,42,0,9,0,5,0,0,0,3,1,0,0,8,0,0,0,0,2,0,2,0,6,1,347,0,7,17,27,0,1,11,80,0,0,0,1,1,11,0,1,37,128
2,288,6,0,-1,z57,Weber County Unincorporated,95,49,057,210101,1,490572101011,Block Group 1,G5030,S,86898873.0,19742.0,+41.3700825,-111.9334158,21478.059645,20832.493173,0.969943,"{'rings': [[[-111.95704997693178, 41.433486063...",95,"{'rings': [[[-111.975799, 41.384468999999996],...",1918,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,2,2,0,0,0,0,0,7,0,0,0,0,0,0,0,0,2,0,2,0,18,0,0,0,0,2,0,0,1,3,2,1,1,0,0,0,0,2,0,0,2,0,0,0,1,1,1,0,0,0,0,0,0,0,0,2,0,1,0,2,1,84,3,0,0,0,6,2,0,0,2,0,0,7,0,0,0,1,0,5,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,2,1,0,0,4,0,2,8,1,0,0,1,2,0,1,4,3,0,12,2,68,0,5,0,0,0,0,1,1,8,6,6,0,4,0,0,1,4,42,0,9,0,5,0,0,0,3,1,0,0,8,0,0,0,0,2,0,2,0,6,1,347,0,7,17,27,0,1,11,80,0,0,0,1,1,11,0,1,37,128
3,2156,90,0,55,NOG,North Ogden,95,49,057,210101,1,490572101011,Block Group 1,G5030,S,86898873.0,19742.0,+41.3700825,-111.9334158,21478.059645,641.857708,0.029884,"{'rings': [[[-111.93408914365247, 41.317315160...",95,"{'rings': [[[-111.975799, 41.384468999999996],...",1918,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,0,0,1,

In [48]:
my_list = list(sdf_LEHDCommutePatterns_joinCity)
my_list

['FID_x',
 'FID_Munici',
 'Id',
 'FID_Muni_1',
 'CODE3',
 'FIRST_NAME',
 'FID_tl_202',
 'STATEFP',
 'COUNTYFP',
 'TRACTCE',
 'BLKGRPCE',
 'GEOID',
 'NAMELSAD',
 'MTFCC',
 'FUNCSTAT',
 'ALAND',
 'AWATER',
 'INTPTLAT',
 'INTPTLON',
 'BGAcres',
 'PieceAcres',
 'PortionBG',
 'SHAPE_x',
 'FID_y',
 'SHAPE_y',
 'fakeid',
 'AFK_x',
 'ALA_x',
 'ALP_x',
 'BDL_x',
 'BGM_x',
 'BNT_x',
 'BRT_x',
 'CDF_x',
 'CEN_x',
 'CHA_x',
 'CHL_x',
 'CLF_x',
 'CLI_x',
 'CMT_x',
 'COA_x',
 'CWH_x',
 'DAN_x',
 'DRA_x',
 'EAG_x',
 'ELK_x',
 'EMT_x',
 'FAR_x',
 'FCS_x',
 'FFD_x',
 'FRR_x',
 'FTH_x',
 'GLA_x',
 'GOS_x',
 'GRL_x',
 'HAR_x',
 'HDT_x',
 'HEB_x',
 'HER_x',
 'HGH_x',
 'HNF_x',
 'HOL_x',
 'HOO_x',
 'HVL_x',
 'IND_x',
 'INT_x',
 'KAY_x',
 'KMS_x',
 'KMT_x',
 'LAY_x',
 'LEH_x',
 'LIN_x',
 'MAP_x',
 'MID_x',
 'MLC_x',
 'MMT_x',
 'MRG_x',
 'MSL_x',
 'MUR_x',
 'MWY_x',
 'NOG_x',
 'NSL_x',
 'OGD_x',
 'OKL_x',
 'ORM_x',
 'PAY_x',
 'PGR_x',
 'PLN_x',
 'PRK_x',
 'PRY_x',
 'PVO_x',
 'PVW_x',
 'ROY_x',
 'RVD_x',
 'RV

In [49]:
#for i in dColCodes:
#    display(sdf_LEHDCommutePatterns_joinCity[[i]])

In [50]:
df_LEHDCommutePatterns_CityCalcs = sdf_LEHDCommutePatterns_joinCity.copy()

for i in dColCodesWCounty:
    display(i)
    
    df_LEHDCommutePatterns_CityCalcs[i] = df_LEHDCommutePatterns_CityCalcs[i] * df_LEHDCommutePatterns_CityCalcs['PortionBG']


'AFK_x'

'ALA_x'

'ALP_x'

'BDL_x'

'BGM_x'

...

'z49_y'

'z51_x'

'z51_y'

'z57_x'

'z57_y'

In [51]:
df_LEHDCommutePatterns_CityCalcs

,FID_x,FID_Munici,Id,FID_Muni_1,CODE3,FIRST_NAME,FID_tl_202,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,BGAcres,PieceAcres,PortionBG,SHAPE_x,FID_y,SHAPE_y,fakeid,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,0,0,0,-1,z05,Cache County Unincorporated,95,49,057,210101,1,490572101011,Block Group 1,G5030,S,86898873.0,19742.0,+41.3700825,-111.9334158,21478.059645,2.821747,0.000131,"{'rings': [[[-111.88525694249688, 41.426031954...",95,"{'rings': [[[-111.975799, 41.384468999999996],...",1918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000526,0.000263,0.0,0.0,0.0,0.0,0.000131,0.0,0.0,0.0,0.000131,0.0,0.0,0.000131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000131,0.0,0.0,0.0,0.000263,0.000263,0.0,0.0,0.0,0.0,0.0,0.000920,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000263,0.0,0.000263,0.0,0.002365,0.0,0.0,0.0,0.0,0.000263,0.0,0.0,0.000131,0.000394,0.000263,0.000131,0.000131,0.0,0.0,0.0,0.0,0.000263,0.0,0.0,0.000263,0.0,0.0,0.0,0.000131,0.000131,0.000131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000263,0.0,0.000131,0.0,0.000263,0.000131,0.011036,0.000394,0.0,0.0,0.0,0.000788,0.000263,0.0,0.0,0.000263,0.0,0.0,0.000920,0.0,0.0,0.0,0.000131,0.0,0.000657,0.0,0.0,0.0,0.000131,0.0,0.0,0.000131,0.0,0.0,0.0,0.0,0.000131,0.0,0.000131,0.0,0.0,0.0,0.0,0.000263,0.000131,0.0,0.0,0.000526,0.0,0.000263,0.001051,0.000131,0.0,0.0,0.000131,0.000263,0.0,0.000131,0.000526,0.000394,0.0,0.001577,0.000263,0.008934,0.0,0.000657,0.0,0.0,0.0,0.0,0.000131,0.000131,0.001051,0.000788,0.000788,0.0,0.000526,0.0,0.0,0.000131,0.000526,0.005518,0.0,0.001182,0.0,0.000657,0.0,0.0,0.0,0.000394,0.000131,0.0,0.0,0.001051,0.0,0.0,0.0,0.0,0.000263,0.0,0.000263,0.0,0.000788,0.000131,0.045588,0.0,0.000920,0.002233,0.003547,0.0,0.000131,0.001445,0.010510,0.0,0.0,0.0,0.000131,0.000131,0.001445,0.0,0.000131,0.004861,0.016816
1,238,5,0,-1,z03,Box Elder County Unincorporated,95,49,057,210101,1,490572101011,Block Group 1,G5030,S,86898873.0,19742.0,+41.3700825,-111.9334158,21478.059645,0.887017,0.000041,"{'rings': [[[-111.95909781387314, 41.433683592...",95,"{'rings': [[[-111.975799, 41.384468999999996],...",1918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000165,0.000083,0.0,0.0,0.0,0.0,0.000041,0.0,0.0,0.0,0.000041,0.0,0.0,0.000041,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000041,0.0,0.0,0.0,0.000083,0.000083,0.0,0.0,0.0,0.0,0.0,0.000289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000083,0.0,0.000083,0.0,0.000743,0.0,0.0,0.0,0.0,0.000083,0.0,0.0,0.000041,0.000124,0.000083,0.000041,0.000041,0.0,0.0,0.0,0.0,0.000083,0.0,0.0,0.000083,0.0,0.0,0.0,0.000041,0.000041,0.000041,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000083,0.0,0.000041,0.0,0.000083,0.000041,0.003469,0.000124,0.0,0.0,0.0,0.000248,0.000083,0.0,0.0,0.000083,0.0,0.0,0.000289,0.0,0.0,0.0,0.000041,0.0,0.000206,0.0,0.0,0

In [52]:
df_LEHDCommutePatterns_CityCalcsSum = df_LEHDCommutePatterns_CityCalcs.groupby(['CODE3','FIRST_NAME'], as_index=False)[dColCodesWCounty].apply(lambda x : x.astype(int).sum())
df_LEHDCommutePatterns_CityCalcsSum = df_LEHDCommutePatterns_CityCalcsSum.reset_index()
df_LEHDCommutePatterns_CityCalcsSum

,CODE3,FIRST_NAME,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,AFK,American Fork,1679,0,286,126,43,95,1,7,34,2,312,46,40,2,2,91,1,449,756,20,3,36,0,1,7,6,8,2,35,7,1,86,257,550,0,75,10,0,0,0,63,3,76,156,1992,319,132,94,124,88,5,1,139,19,23,19,135,1,2307,202,1453,8,6,5,1312,9,61,9,364,432,115,788,499,406,379,76,28,506,54,6,10,36,155,132,0,2,18,0,3,2,18,12,3,427,13,311,20,21026,1689,3,107,47,8,19,1,1,9,1,29,8,3,0,1,104,1,441,34,0,0,8,0,0,0,0,0,0,1,0,0,33,37,169,0,16,3,0,0,0,3,0,5,51,1379,531,17,73,114,1,0,0,197,8,3,32,81,1,1352,28,596,0,12,1,1375,0,15,13,39,280,9,108,142,205,797,11,5,176,115,0,0,4,66,15,0,21,1,0,0,0,1,3,1,119,0,212,7,11975,58,10,694,207,5,0,4277,3046,25,16,178,16,13544,7953,134,47,406,161
1,ALA,Alta,1,2,0,0,0,0,1,0,0,0,0,0,0,0,0,40,0,8,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,14,0,0,0,0,0,0,2,0,1,0,0,13,19,1,0,0,11,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,3,42,0,0,0,6,41,0,0,0,5,0,0,0,7,1,0,0,0,0,0,0,2,0,0,11,0,7,0,297,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,3,3,0,0,0,6,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,9,0,0,0,2,32,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,5,0,115,0,0,6,3,0,0,252,93,0,2,1,0,12,5,1,0,1,0
2,ALP,Alpine,114,0,302,10,3,11,0,0,2,0,57,1,5,0,0,21,1,33,80,2,2,2,0,0,1,1,2,0,84,0,0,0,21,129,1,13,2,0,0,0,1,0,6,14,163,31,7,3,34,7,2,0,17,2,1,3,14,0,195,12,117,1,1,0,109,0,10,0,33,63,13,53,22,37,30,7,4,33,6,1,1,2,14,268,0,0,2,0,1,1,6,2,0,25,1,26,4,2682,273,0,273,11,1,10,0,0,1,0,18,4,1,4,0,54,0,135,10,0,0,2,0,0,0,0,0,0,3,1,0,8,5,131,0,25,0,0,0,0,7,2,1,16,427,138,7,46,65,3,0,1,116,2,2,10,35,0,296,9,168,1,18,0,341,1,4,4,24,140,1,23,20,88,403,6,2,30,64,0,1,4,45,8,0,1,0,0,0,1,0,3,0,55,0,111,2,4137,3,1,64,79,2,0,457,1468,3,21,354,13,1504,2228,5,11,43,69
3,BDL,Bluffdale,50,0,11,394,3,17,0,4,4,0,10,9,4,3,0,26,1,198,133,0,0,5,0,1,0,0,1,1,9,1,0,11,163,30,0,26,0,0,0,0,1,0,80,23,159,12,6,64,65,33,0,0,78,1,3,12,29,0,101,18,27,3,0,0,84,0,14,2,296,228,14,84,37,179,132,10,2,24,23,0,2,8,128,25,0,0,1,0,1,0,8,4,0,277,1,181,1,4093,139,0,9,386,0,21,0,0,15,0,1,5,1,0,0,147,0,574,13,0,0,0,0,0,0,0,0,0,1,0,0,1,51,5,0,50,0,0,0,0,2,0,18,14,294,50,0,175,131,11,0,0,298,0,0,30,27,0,206,4,26,0,5,0,121,0,3,6,227,459,0,10,10,368,1084,0,2,10,180,0,0,2,104,7,0,1,0,0,0,7,0,0,0,350,1,310,7,6335,3,0,123,142,0,0,2670,5006,3,5,38,9,900,949,21,3,81,56
4,BGM,Brigham City,6,0,2,2,2282,18,0,0,5,0,5,55,28,0,2,9,0,13,4,1,1,6,0,0,30,5,2,1,3,41,1,9,3,6,2,13,23,2,0,0,22,2,16,86,13,4,2,7,11,7,9,8,16,1,146,10,467,1,24,4,9,31,4,459,26,80,148,32,13,27,3,6,10,13,77,2,53,15,9,8,4,33,14,27,3,0,35,0,0,3,3,34,92,24,15,46,11,7081,48,0,4,7,2452,50,0,0,24,0,3,159,19,0,2,27,0,72,4,0,1,29,0,0,76,1,0,0,5,28,0,15,5,6,0,21,4,0,0,0,34,5,9,208,39,21,2,34,65,6,6,32,111,1,69,76,1028,0,94,15,21,17,10,219,89,50,92,99,24,87,1,7,33,56,5

In [53]:
df_LEHDCommutePatterns_CityCalcsSum.sum()

CODE3         AFKALAALPBDLBGMBNTBRTCDFCENCHACHLCLFCLICMTCOAC...
FIRST_NAME    American ForkAltaAlpineBluffdaleBrigham CityBo...
AFK_x                                                     11728
ALA_x                                                       114
ALP_x                                                      4033
                                    ...                        
z49_y                                                    234000
z51_x                                                     11254
z51_y                                                      6750
z57_x                                                    107383
z57_y                                                     99203
Length: 216, dtype: object

In [54]:
sdf_LEHDCommutePatterns_joinCity_Percent_SA = pd.DataFrame.merge(sdf_CityTownshipCounty_BlockGroups,sdf_LEHDCommutePatterns_Percent_SA,on=['GEOID','STATEFP','COUNTYFP','TRACTCE','BLKGRPCE','NAMELSAD','MTFCC', 'FUNCSTAT','ALAND','AWATER','INTPTLAT','INTPTLON','BGAcres'])

df_LEHDCommutePatterns_CityCalcs_Percent_SA = sdf_LEHDCommutePatterns_joinCity_Percent_SA.copy()

for i in dColCodesWCounty:    
    df_LEHDCommutePatterns_CityCalcs_Percent_SA[i] = df_LEHDCommutePatterns_CityCalcs_Percent_SA[i] * df_LEHDCommutePatterns_CityCalcs_Percent_SA['PortionBG']


display(df_LEHDCommutePatterns_CityCalcs_Percent_SA.sum())
    
df_LEHDCommutePatterns_CityCalcs_Percent_SA = df_LEHDCommutePatterns_CityCalcs_Percent_SA.fillna(0)
   

df_LEHDCommutePatterns_CityCalcsSum_Percent_SA = df_LEHDCommutePatterns_CityCalcs_Percent_SA.groupby(['CODE3','FIRST_NAME'], as_index=False)[dColCodesWCounty].apply(lambda x : x.astype(float).sum())
df_LEHDCommutePatterns_CityCalcsSum_Percent_SA = df_LEHDCommutePatterns_CityCalcsSum_Percent_SA.reset_index()
df_LEHDCommutePatterns_CityCalcsSum_Percent_SA
    
display(df_LEHDCommutePatterns_CityCalcsSum_Percent_SA.sum())

FID_x                                                   6571649
FID_Munici                                               250090
Id                                                            0
FID_Muni_1                                               139357
CODE3         z05z03z57NOGz05z05z05z05z05z05z05z05z05z05z03z...
                                    ...                        
z49_y                                                   99.9999
z51_x                                                       100
z51_y                                                   99.9999
z57_x                                                       100
z57_y                                                   99.9998
Length: 238, dtype: object

CODE3         AFKALAALPBDLBGMBNTBRTCDFCENCHACHLCLFCLICMTCOAC...
FIRST_NAME    American ForkAltaAlpineBluffdaleBrigham CityBo...
AFK_x                                                       100
ALA_x                                                       100
ALP_x                                                       100
                                    ...                        
z49_y                                                   99.9999
z51_x                                                       100
z51_y                                                   99.9999
z57_x                                                       100
z57_y                                                   99.9998
Length: 216, dtype: object

In [55]:
sdf_LEHDCommutePatterns_City_Number = pd.DataFrame.merge(sdf_CityTownshipWCodes,df_LEHDCommutePatterns_CityCalcsSum,on=['CODE3','FIRST_NAME'])
sdf_LEHDCommutePatterns_City_Percent_SA = pd.DataFrame.merge(sdf_CityTownshipWCodes,df_LEHDCommutePatterns_CityCalcsSum_Percent_SA,on=['CODE3','FIRST_NAME'])

In [56]:
sdf_LEHDCommutePatterns_City_Number.spatial.to_featureclass(fname_CitiesWCommutePatterns_Number)

'E:\\GitHub\\Census-LEHD-Commute-Patterns\\results\\LEHDCommutePatterns.gdb\\City_CommutePatterns_Number'

In [57]:
sdf_LEHDCommutePatterns_City_Percent_SA.spatial.to_featureclass(fname_CitiesWCommutePatterns_Percent_SA)

'E:\\GitHub\\Census-LEHD-Commute-Patterns\\results\\LEHDCommutePatterns.gdb\\City_CommutePatterns_Percent_SelectedArea'

In [58]:
sdf_LEHDCommutePatterns_City_Number

,OBJECTID,CODE3,FIRST_NAME,FIRST_COUNTYNBR,SHAPE,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,1,AFK,American Fork,25,"{""rings"": [[[-111.77435121873373, 40.353624868...",1679,0,286,126,43,95,1,7,34,2,312,46,40,2,2,91,1,449,756,20,3,36,0,1,7,6,8,2,35,7,1,86,257,550,0,75,10,0,0,0,63,3,76,156,1992,319,132,94,124,88,5,1,139,19,23,19,135,1,2307,202,1453,8,6,5,1312,9,61,9,364,432,115,788,499,406,379,76,28,506,54,6,10,36,155,132,0,2,18,0,3,2,18,12,3,427,13,311,20,21026,1689,3,107,47,8,19,1,1,9,1,29,8,3,0,1,104,1,441,34,0,0,8,0,0,0,0,0,0,1,0,0,33,37,169,0,16,3,0,0,0,3,0,5,51,1379,531,17,73,114,1,0,0,197,8,3,32,81,1,1352,28,596,0,12,1,1375,0,15,13,39,280,9,108,142,205,797,11,5,176,115,0,0,4,66,15,0,21,1,0,0,0,1,3,1,119,0,212,7,11975,58,10,694,207,5,0,4277,3046,25,16,178,16,13544,7953,134,47,406,161
1,2,ALA,Alta,18,"{""rings"": [[[-111.59946851855727, 40.577431980...",1,2,0,0,0,0,1,0,0,0,0,0,0,0,0,40,0,8,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,14,0,0,0,0,0,0,2,0,1,0,0,13,19,1,0,0,11,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,3,42,0,0,0,6,41,0,0,0,5,0,0,0,7,1,0,0,0,0,0,0,2,0,0,11,0,7,0,297,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,3,3,0,0,0,6,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,9,0,0,0,2,32,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,5,0,115,0,0,6,3,0,0,252,93,0,2,1,0,12,5,1,0,1,0
2,3,ALP,Alpine,25,"{""rings"": [[[-111.75455302160464, 40.479513910...",114,0,302,10,3,11,0,0,2,0,57,1,5,0,0,21,1,33,80,2,2,2,0,0,1,1,2,0,84,0,0,0,21,129,1,13,2,0,0,0,1,0,6,14,163,31,7,3,34,7,2,0,17,2,1,3,14,0,195,12,117,1,1,0,109,0,10,0,33,63,13,53,22,37,30,7,4,33,6,1,1,2,14,268,0,0,2,0,1,1,6,2,0,25,1,26,4,2682,273,0,273,11,1,10,0,0,1,0,18,4,1,4,0,54,0,135,10,0,0,2,0,0,0,0,0,0,3,1,0,8,5,131,0,25,0,0,0,0,7,2,1,16,427,138,7,46,65,3,0,1,116,2,2,10,35,0,296,9,168,1,18,0,341,1,4,4,24,140,1,23,20,88,403,6,2,30,64,0,1,4,45,8,0,1,0,0,0,1,0,3,0,55,0,111,2,4137,3,1,64,79,2,0,457,1468,3,21,354,13,1504,2228,5,11,43,69
3,4,BDL,Bluffdale,18,"{""rings"": [[[-111.9704071902396, 40.5008379065...",50,0,11,394,3,17,0,4,4,0,10,9,4,3,0,26,1,198,133,0,0,5,0,1,0,0,1,1,9,1,0,11,163,30,0,26,0,0,0,0,1,0,80,23,159,12,6,64,65,33,0,0,78,1,3,12,29,0,101,18,27,3,0,0,84,0,14,2,296,228,14,84,37,179,132,10,2,24,23,0,2,8,128,25,0,0,1,0,1,0,8,4,0,277,1,181,1,4093,139,0,9,386,0,21,0,0,15,0,1,5,1,0,0,147,0,574,13,0,0,0,0,0,0,0,0,0,1,0,0,1,51,5,0,50,0,0,0,0,2,0,18,14,294,50,0,175,131,11,0,0,298,0,0,30,27,0,206,4,26,0,5,0,121,0,3,6,227,459,0,10,10,368,1084,0,2,10,180,0,0,2,104,7,0,1,0,0,0,7,0,0,0,350,1,310,7,6335,3,0,123,142,0,0,2670,5006,3,5,38,9,900,949,21,3,81,56
4,5,BGM,Brigham City,02,"{""rings"": [[[-112.01898409573937, 41.535781784...",6,0,2,2,2282,18,0,0,5,0,5,55,28,0,2,9,0,13,4,1,1,6,0,0,30,5,2,1,3,41,1,9,3,6,2,13,23,2,0,0,22,2,16,86,13,4,2,7,11,7,

# Calculate Percents by Map Units

In [59]:
# prepare column list

#display(dColCodesWCounty)
dColCodesWCounty = dColCodesWCounty.copy()

dColCodesWCounty_x = [x for x in dColCodesWCounty if "y" not in x]
dColCodesWCounty_y = [x for x in dColCodesWCounty if "x" not in x]

#remove and append to make sure at end
dColCodesWCounty_x.remove('TTL_x')
dColCodesWCounty_y.remove('TTL_y')
dColCodesWCounty_x.append('TTL_x')
dColCodesWCounty_y.append('TTL_y')

display(dColCodesWCounty_x)
display(dColCodesWCounty_y)

dColCodesNoTotal_x = [x for x in dColCodes if "y" not in x]
dColCodesNoTotal_x.remove('TTL_x')

dColCodesNoTotal_x = [x for x in dColCodes if "x" not in x]
dColCodesNoTotal_x.remove('TTL_y')

['AFK_x',
 'ALA_x',
 'ALP_x',
 'BDL_x',
 'BGM_x',
 'BNT_x',
 'BRT_x',
 'CDF_x',
 'CEN_x',
 'CHA_x',
 'CHL_x',
 'CLF_x',
 'CLI_x',
 'CMT_x',
 'COA_x',
 'CWH_x',
 'DAN_x',
 'DRA_x',
 'EAG_x',
 'ELK_x',
 'EMT_x',
 'FAR_x',
 'FCS_x',
 'FFD_x',
 'FRR_x',
 'FTH_x',
 'GLA_x',
 'GOS_x',
 'GRL_x',
 'HAR_x',
 'HDT_x',
 'HEB_x',
 'HER_x',
 'HGH_x',
 'HNF_x',
 'HOL_x',
 'HOO_x',
 'HVL_x',
 'IND_x',
 'INT_x',
 'KAY_x',
 'KMS_x',
 'KMT_x',
 'LAY_x',
 'LEH_x',
 'LIN_x',
 'MAP_x',
 'MID_x',
 'MLC_x',
 'MMT_x',
 'MRG_x',
 'MSL_x',
 'MUR_x',
 'MWY_x',
 'NOG_x',
 'NSL_x',
 'OGD_x',
 'OKL_x',
 'ORM_x',
 'PAY_x',
 'PGR_x',
 'PLN_x',
 'PRK_x',
 'PRY_x',
 'PVO_x',
 'PVW_x',
 'ROY_x',
 'RVD_x',
 'RVT_x',
 'SAN_x',
 'SAQ_x',
 'SAR_x',
 'SFK_x',
 'SJC_x',
 'SLC_x',
 'SLM_x',
 'SOG_x',
 'SPV_x',
 'SSL_x',
 'SUN_x',
 'SWE_x',
 'SYR_x',
 'TAY_x',
 'TOO_x',
 'UIN_x',
 'VIN_x',
 'WAT_x',
 'WBG_x',
 'WDL_x',
 'WEB_x',
 'WHT_x',
 'WHV_x',
 'WIL_x',
 'WJC_x',
 'WPT_x',
 'WVC_x',
 'WXC_x',
 'z03_x',
 'z11_x',
 'z29_x',


['AFK_y',
 'ALA_y',
 'ALP_y',
 'BDL_y',
 'BGM_y',
 'BNT_y',
 'BRT_y',
 'CDF_y',
 'CEN_y',
 'CHA_y',
 'CHL_y',
 'CLF_y',
 'CLI_y',
 'CMT_y',
 'COA_y',
 'CWH_y',
 'DAN_y',
 'DRA_y',
 'EAG_y',
 'ELK_y',
 'EMT_y',
 'FAR_y',
 'FCS_y',
 'FFD_y',
 'FRR_y',
 'FTH_y',
 'GLA_y',
 'GOS_y',
 'GRL_y',
 'HAR_y',
 'HDT_y',
 'HEB_y',
 'HER_y',
 'HGH_y',
 'HNF_y',
 'HOL_y',
 'HOO_y',
 'HVL_y',
 'IND_y',
 'INT_y',
 'KAY_y',
 'KMS_y',
 'KMT_y',
 'LAY_y',
 'LEH_y',
 'LIN_y',
 'MAP_y',
 'MID_y',
 'MLC_y',
 'MMT_y',
 'MRG_y',
 'MSL_y',
 'MUR_y',
 'MWY_y',
 'NOG_y',
 'NSL_y',
 'OGD_y',
 'OKL_y',
 'ORM_y',
 'PAY_y',
 'PGR_y',
 'PLN_y',
 'PRK_y',
 'PRY_y',
 'PVO_y',
 'PVW_y',
 'ROY_y',
 'RVD_y',
 'RVT_y',
 'SAN_y',
 'SAQ_y',
 'SAR_y',
 'SFK_y',
 'SJC_y',
 'SLC_y',
 'SLM_y',
 'SOG_y',
 'SPV_y',
 'SSL_y',
 'SUN_y',
 'SWE_y',
 'SYR_y',
 'TAY_y',
 'TOO_y',
 'UIN_y',
 'VIN_y',
 'WAT_y',
 'WBG_y',
 'WDL_y',
 'WEB_y',
 'WHT_y',
 'WHV_y',
 'WIL_y',
 'WJC_y',
 'WPT_y',
 'WVC_y',
 'WXC_y',
 'z03_y',
 'z11_y',
 'z29_y',


In [60]:
# block group
sdf_LEHDCommutePatterns_Percent_MU = sdf_LEHDCommutePatterns.copy()
sdf_LEHDCommutePatterns_tract_Percent_MU  = sdf_LEHDCommutePatterns_tract.copy()
sdf_LEHDCommutePatterns_City_Percent_MU  = sdf_LEHDCommutePatterns_City_Number.copy()

for x in dColCodesWCounty_x:   
    #display(x)
    sdf_LEHDCommutePatterns_Percent_MU      [x] = sdf_LEHDCommutePatterns_Percent_MU      [x] / sdf_LEHDCommutePatterns_Percent_MU      ['TTL_x'] * 100
    sdf_LEHDCommutePatterns_tract_Percent_MU[x] = sdf_LEHDCommutePatterns_tract_Percent_MU[x] / sdf_LEHDCommutePatterns_tract_Percent_MU['TTL_x'] * 100
    sdf_LEHDCommutePatterns_City_Percent_MU [x] = sdf_LEHDCommutePatterns_City_Percent_MU [x] / sdf_LEHDCommutePatterns_City_Percent_MU ['TTL_x'] * 100

for y in dColCodesWCounty_y:   
    #display(y)
    sdf_LEHDCommutePatterns_Percent_MU      [y] = sdf_LEHDCommutePatterns_Percent_MU      [y] / sdf_LEHDCommutePatterns_Percent_MU      ['TTL_y'] * 100
    sdf_LEHDCommutePatterns_tract_Percent_MU[y] = sdf_LEHDCommutePatterns_tract_Percent_MU[y] / sdf_LEHDCommutePatterns_tract_Percent_MU['TTL_y'] * 100
    sdf_LEHDCommutePatterns_City_Percent_MU [y] = sdf_LEHDCommutePatterns_City_Percent_MU [y] / sdf_LEHDCommutePatterns_City_Percent_MU ['TTL_y'] * 100
    

In [61]:
#check that everything is near 100 (subtracting out non-city areas)
display(sdf_LEHDCommutePatterns_Percent_MU      .loc[:,dColCodesNoTotal_x].sum(axis=1))
display(sdf_LEHDCommutePatterns_tract_Percent_MU.loc[:,dColCodesNoTotal_x].sum(axis=1))
display(sdf_LEHDCommutePatterns_City_Percent_MU .loc[:,dColCodesNoTotal_x].sum(axis=1))

0       96.056623
1       95.652174
2       97.188049
3       97.471910
4       97.443182
          ...    
2014    94.126506
2015    93.772242
2016    92.699115
2017    92.105263
2018    92.774566
Length: 2019, dtype: float64

0      96.342902
1      95.646842
2      92.147686
3      96.082546
4      96.892265
         ...    
711    96.519807
712    96.460573
713    96.696697
714    97.612958
715    20.242537
Length: 716, dtype: float64

0     91.841336
1     74.782609
2     89.968576
3     94.380426
4     79.529277
        ...    
92    64.492754
93    95.149523
94    88.479516
95    95.629469
96    89.995752
Length: 97, dtype: float64

In [62]:
#export to GDB
deleteIfExists(fname_BlockGroupsWCommutePatterns_Percent_MU)
deleteIfExists(fname_TractsWCommutePatterns_Percent_MU     )
deleteIfExists(fname_CitiesWCommutePatterns_Percent_MU     )

sdf_LEHDCommutePatterns_Percent_MU      .spatial.to_featureclass(fname_BlockGroupsWCommutePatterns_Percent_MU)
sdf_LEHDCommutePatterns_tract_Percent_MU.spatial.to_featureclass(fname_TractsWCommutePatterns_Percent_MU     )
sdf_LEHDCommutePatterns_City_Percent_MU .spatial.to_featureclass(fname_CitiesWCommutePatterns_Percent_MU     )

'E:\\GitHub\\Census-LEHD-Commute-Patterns\\results\\LEHDCommutePatterns.gdb\\City_CommutePatterns_Percent_MapUnit'

# Summarize by County

In [63]:
df_CityTownship_NameCode

,NAME,CODE3,zCOFIPS
0,Alpine,ALP,z49
1,American Fork,AFK,z49
2,Cedar Fort,CDF,z49
3,Cedar Hills,CHL,z49
4,Eagle Mountain,EAG,z49
...,...,...,...
92,Washington Terrace,WAT,z57
93,West Haven,WHV,z57
94,Grantsville,GRL,z45
95,Tooele,TOO,z45


In [64]:
display(df_LEHDCommutePatterns_CityCalcsSum)
display(df_LEHDCommutePatterns_CityCalcsSum_Percent_SA)

,CODE3,FIRST_NAME,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,AFK,American Fork,1679,0,286,126,43,95,1,7,34,2,312,46,40,2,2,91,1,449,756,20,3,36,0,1,7,6,8,2,35,7,1,86,257,550,0,75,10,0,0,0,63,3,76,156,1992,319,132,94,124,88,5,1,139,19,23,19,135,1,2307,202,1453,8,6,5,1312,9,61,9,364,432,115,788,499,406,379,76,28,506,54,6,10,36,155,132,0,2,18,0,3,2,18,12,3,427,13,311,20,21026,1689,3,107,47,8,19,1,1,9,1,29,8,3,0,1,104,1,441,34,0,0,8,0,0,0,0,0,0,1,0,0,33,37,169,0,16,3,0,0,0,3,0,5,51,1379,531,17,73,114,1,0,0,197,8,3,32,81,1,1352,28,596,0,12,1,1375,0,15,13,39,280,9,108,142,205,797,11,5,176,115,0,0,4,66,15,0,21,1,0,0,0,1,3,1,119,0,212,7,11975,58,10,694,207,5,0,4277,3046,25,16,178,16,13544,7953,134,47,406,161
1,ALA,Alta,1,2,0,0,0,0,1,0,0,0,0,0,0,0,0,40,0,8,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,14,0,0,0,0,0,0,2,0,1,0,0,13,19,1,0,0,11,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,3,42,0,0,0,6,41,0,0,0,5,0,0,0,7,1,0,0,0,0,0,0,2,0,0,11,0,7,0,297,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,3,3,0,0,0,6,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,9,0,0,0,2,32,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,5,0,115,0,0,6,3,0,0,252,93,0,2,1,0,12,5,1,0,1,0
2,ALP,Alpine,114,0,302,10,3,11,0,0,2,0,57,1,5,0,0,21,1,33,80,2,2,2,0,0,1,1,2,0,84,0,0,0,21,129,1,13,2,0,0,0,1,0,6,14,163,31,7,3,34,7,2,0,17,2,1,3,14,0,195,12,117,1,1,0,109,0,10,0,33,63,13,53,22,37,30,7,4,33,6,1,1,2,14,268,0,0,2,0,1,1,6,2,0,25,1,26,4,2682,273,0,273,11,1,10,0,0,1,0,18,4,1,4,0,54,0,135,10,0,0,2,0,0,0,0,0,0,3,1,0,8,5,131,0,25,0,0,0,0,7,2,1,16,427,138,7,46,65,3,0,1,116,2,2,10,35,0,296,9,168,1,18,0,341,1,4,4,24,140,1,23,20,88,403,6,2,30,64,0,1,4,45,8,0,1,0,0,0,1,0,3,0,55,0,111,2,4137,3,1,64,79,2,0,457,1468,3,21,354,13,1504,2228,5,11,43,69
3,BDL,Bluffdale,50,0,11,394,3,17,0,4,4,0,10,9,4,3,0,26,1,198,133,0,0,5,0,1,0,0,1,1,9,1,0,11,163,30,0,26,0,0,0,0,1,0,80,23,159,12,6,64,65,33,0,0,78,1,3,12,29,0,101,18,27,3,0,0,84,0,14,2,296,228,14,84,37,179,132,10,2,24,23,0,2,8,128,25,0,0,1,0,1,0,8,4,0,277,1,181,1,4093,139,0,9,386,0,21,0,0,15,0,1,5,1,0,0,147,0,574,13,0,0,0,0,0,0,0,0,0,1,0,0,1,51,5,0,50,0,0,0,0,2,0,18,14,294,50,0,175,131,11,0,0,298,0,0,30,27,0,206,4,26,0,5,0,121,0,3,6,227,459,0,10,10,368,1084,0,2,10,180,0,0,2,104,7,0,1,0,0,0,7,0,0,0,350,1,310,7,6335,3,0,123,142,0,0,2670,5006,3,5,38,9,900,949,21,3,81,56
4,BGM,Brigham City,6,0,2,2,2282,18,0,0,5,0,5,55,28,0,2,9,0,13,4,1,1,6,0,0,30,5,2,1,3,41,1,9,3,6,2,13,23,2,0,0,22,2,16,86,13,4,2,7,11,7,9,8,16,1,146,10,467,1,24,4,9,31,4,459,26,80,148,32,13,27,3,6,10,13,77,2,53,15,9,8,4,33,14,27,3,0,35,0,0,3,3,34,92,24,15,46,11,7081,48,0,4,7,2452,50,0,0,24,0,3,159,19,0,2,27,0,72,4,0,1,29,0,0,76,1,0,0,5,28,0,15,5,6,0,21,4,0,0,0,34,5,9,208,39,21,2,34,65,6,6,32,111,1,69,76,1028,0,94,15,21,17,10,219,89,50,92,99,24,87,1,7,33,56,5

,CODE3,FIRST_NAME,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,AFK,American Fork,13.745828,1.154077,6.647175,2.086242,0.544271,0.519737,1.315789,3.691058e+00,0.606382,2.160822,7.299090,0.426051,0.487027,0.972586,0.865667,0.581010,1.440952,2.064437,5.263535,1.789345,0.387111,4.907119e-01,0.595423,4.704245,3.053334e-01,0.356874,1.731732,1.299778,0.854792,0.349247,1.341870,1.167039,1.574970,7.186896,0.813910,0.565606,0.417439,0.412435,0.821956,0.752218,0.568524,0.898416,0.462113,0.522082,6.890394,6.511045e+00,3.106985,0.612327,0.433357,0.666584,0.513489,0.278380,0.578926,1.330064,0.326626,3.354154e-01,0.370843,0.713487,5.215473,2.212396,9.194530,5.036920e-01,0.286759,0.360446,3.335315,0.331106,3.907812e-01,0.364066,1.581521,0.911951,2.239952e+00,5.798792,2.587612,1.273501,0.416138,2.138050e+00,0.458050,3.102331,0.549505,0.511081,0.493644,3.766680e-01,0.534668,0.847345,0.270838,5.349500,0.583988,0.476385,1.155429,2.335817e-01,0.804262,0.381554,0.667880,0.737498,0.408957,0.505505,0.538337,1.490681,7.581500,0.487457,4.196949,1.279297,0.124643,0.265294,1.666667,2.471182,0.264115,0.946878,4.579197,0.077555,0.282283,0.000000,0.164745,0.568363,0.625523,1.197594,2.205569,1.245706,0.691051,1.538886e-01,0.000000,2.722201,3.501203e-02,0.108217,0.000000,3.333612,8.793877e-02,0.122008,0.000000,0.722369,1.064654,5.891179,0.000000,0.218100,0.520303,0.000000,0.000000,0.0,0.087289,0.105193,0.240585,0.229502,4.183504,4.320092,1.304506e+00,0.458487,0.446271,0.193528,0.000325,0.080735,0.473258,0.995585,0.140364,0.275002,0.157218,0.761488,2.864203,0.626639,5.682859,0.000000,0.182563,0.165785,2.374763,0.102946,0.320645,0.267974,0.494943,0.608292,1.578062,3.414142e+00,1.219887e+00,0.725611,0.311439,1.017961e+00,0.143230,1.474433,0.365985,0.082788,0.155423,2.025571e-01,0.374756,0.312888,0.000000,3.120291,0.097746,0.000000,0.532411,0.152280,7.557115e-01,1.854353e-01,1.048607,0.365221,0.141842,0.324514,0.265748,0.849412,0.517652,0.145144,0.481634,0.198265,0.513489,0.000325,0.770076,0.447916,0.516145,0.184484,0.849089,0.272550,5.292383,3.389366,1.222309,0.761407,0.386184,0.169521
1,ALA,Alta,0.012512,1.334277,0.013142,0.009075,0.001992,0.003969,1.736294,0.000000e+00,0.005027,0.000000,0.009028,0.000000,0.003723,0.000000,0.000000,0.236068,0.000000,0.041776,0.002575,0.000003,0.070859,1.226863e-02,0.001914,0.000000,0.000000e+00,0.008897,0.000000,0.000000,0.000164,0.006163,0.057762,0.004849,0.013505,0.010499,0.000000,0.096431,0.000000,0.000000,0.000000,0.000000,0.007443,0.000000,0.015514,0.002989,0.005949,1.141404e-02,0.008332,0.080251,0.062619,0.012993,0.000000,0.000000,0.047574,0.025482,0.002150,4.556772e-03,0.000514,0.000000,0.004254,0.005942,0.008392,0.000000e+00,0.024827,0.000000,0.002903,0.000000,1.998306e-03,0.004636,0.015353,0.086885,6.6

In [65]:
df_CitySum_wCounty_Join            = pd.DataFrame.merge(df_LEHDCommutePatterns_CityCalcsSum           , df_CityTownship_NameCode, on="CODE3", how="left")
df_CitySum_wCounty_Join_Percent_SA = pd.DataFrame.merge(df_LEHDCommutePatterns_CityCalcsSum_Percent_SA, df_CityTownship_NameCode, on="CODE3", how="left")

df_CitySum_wCounty_Join           .loc[df_CitySum_wCounty_Join           .CODE3.str[0] == 'z', 'zCOFIPS'] = df_CitySum_wCounty_Join           .CODE3
df_CitySum_wCounty_Join_Percent_SA.loc[df_CitySum_wCounty_Join_Percent_SA.CODE3.str[0] == 'z', 'zCOFIPS'] = df_CitySum_wCounty_Join_Percent_SA.CODE3

display(df_CitySum_wCounty_Join)
display(df_CitySum_wCounty_Join_Percent_SA)

,CODE3,FIRST_NAME,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y,NAME,zCOFIPS
0,AFK,American Fork,1679,0,286,126,43,95,1,7,34,2,312,46,40,2,2,91,1,449,756,20,3,36,0,1,7,6,8,2,35,7,1,86,257,550,0,75,10,0,0,0,63,3,76,156,1992,319,132,94,124,88,5,1,139,19,23,19,135,1,2307,202,1453,8,6,5,1312,9,61,9,364,432,115,788,499,406,379,76,28,506,54,6,10,36,155,132,0,2,18,0,3,2,18,12,3,427,13,311,20,21026,1689,3,107,47,8,19,1,1,9,1,29,8,3,0,1,104,1,441,34,0,0,8,0,0,0,0,0,0,1,0,0,33,37,169,0,16,3,0,0,0,3,0,5,51,1379,531,17,73,114,1,0,0,197,8,3,32,81,1,1352,28,596,0,12,1,1375,0,15,13,39,280,9,108,142,205,797,11,5,176,115,0,0,4,66,15,0,21,1,0,0,0,1,3,1,119,0,212,7,11975,58,10,694,207,5,0,4277,3046,25,16,178,16,13544,7953,134,47,406,161,American Fork,z49
1,ALA,Alta,1,2,0,0,0,0,1,0,0,0,0,0,0,0,0,40,0,8,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,14,0,0,0,0,0,0,2,0,1,0,0,13,19,1,0,0,11,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,3,42,0,0,0,6,41,0,0,0,5,0,0,0,7,1,0,0,0,0,0,0,2,0,0,11,0,7,0,297,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,3,3,0,0,0,6,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,9,0,0,0,2,32,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,5,0,115,0,0,6,3,0,0,252,93,0,2,1,0,12,5,1,0,1,0,Alta,z35
2,ALP,Alpine,114,0,302,10,3,11,0,0,2,0,57,1,5,0,0,21,1,33,80,2,2,2,0,0,1,1,2,0,84,0,0,0,21,129,1,13,2,0,0,0,1,0,6,14,163,31,7,3,34,7,2,0,17,2,1,3,14,0,195,12,117,1,1,0,109,0,10,0,33,63,13,53,22,37,30,7,4,33,6,1,1,2,14,268,0,0,2,0,1,1,6,2,0,25,1,26,4,2682,273,0,273,11,1,10,0,0,1,0,18,4,1,4,0,54,0,135,10,0,0,2,0,0,0,0,0,0,3,1,0,8,5,131,0,25,0,0,0,0,7,2,1,16,427,138,7,46,65,3,0,1,116,2,2,10,35,0,296,9,168,1,18,0,341,1,4,4,24,140,1,23,20,88,403,6,2,30,64,0,1,4,45,8,0,1,0,0,0,1,0,3,0,55,0,111,2,4137,3,1,64,79,2,0,457,1468,3,21,354,13,1504,2228,5,11,43,69,Alpine,z49
3,BDL,Bluffdale,50,0,11,394,3,17,0,4,4,0,10,9,4,3,0,26,1,198,133,0,0,5,0,1,0,0,1,1,9,1,0,11,163,30,0,26,0,0,0,0,1,0,80,23,159,12,6,64,65,33,0,0,78,1,3,12,29,0,101,18,27,3,0,0,84,0,14,2,296,228,14,84,37,179,132,10,2,24,23,0,2,8,128,25,0,0,1,0,1,0,8,4,0,277,1,181,1,4093,139,0,9,386,0,21,0,0,15,0,1,5,1,0,0,147,0,574,13,0,0,0,0,0,0,0,0,0,1,0,0,1,51,5,0,50,0,0,0,0,2,0,18,14,294,50,0,175,131,11,0,0,298,0,0,30,27,0,206,4,26,0,5,0,121,0,3,6,227,459,0,10,10,368,1084,0,2,10,180,0,0,2,104,7,0,1,0,0,0,7,0,0,0,350,1,310,7,6335,3,0,123,142,0,0,2670,5006,3,5,38,9,900,949,21,3,81,56,Bluffdale,z35
4,BGM,Brigham City,6,0,2,2,2282,18,0,0,5,0,5,55,28,0,2,9,0,13,4,1,1,6,0,0,30,5,2,1,3,41,1,9,3,6,2,13,23,2,0,0,22,2,16,86,13,4,2,7,11,7,9,8,16,1,146,10,467,1,24,4,9,31,4,459,26,80,148,32,13,27,3,6,10,13,77,2,53,15,9,8,4,33,14,27,3,0,35,0,0,3,3,34,92,24,15,46,11,7081,48,0,4,7,2452,50,0,0,24,0,3,159,19,0,2,27,0,72,4,0,1,29,0,0,76,1,0,0,5,28,0,15,5,6,0,21,4,0,0,0,34,5,9,208,39,21,2,34,65,6,6,32,11

,CODE3,FIRST_NAME,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y,NAME,zCOFIPS
0,AFK,American Fork,13.745828,1.154077,6.647175,2.086242,0.544271,0.519737,1.315789,3.691058e+00,0.606382,2.160822,7.299090,0.426051,0.487027,0.972586,0.865667,0.581010,1.440952,2.064437,5.263535,1.789345,0.387111,4.907119e-01,0.595423,4.704245,3.053334e-01,0.356874,1.731732,1.299778,0.854792,0.349247,1.341870,1.167039,1.574970,7.186896,0.813910,0.565606,0.417439,0.412435,0.821956,0.752218,0.568524,0.898416,0.462113,0.522082,6.890394,6.511045e+00,3.106985,0.612327,0.433357,0.666584,0.513489,0.278380,0.578926,1.330064,0.326626,3.354154e-01,0.370843,0.713487,5.215473,2.212396,9.194530,5.036920e-01,0.286759,0.360446,3.335315,0.331106,3.907812e-01,0.364066,1.581521,0.911951,2.239952e+00,5.798792,2.587612,1.273501,0.416138,2.138050e+00,0.458050,3.102331,0.549505,0.511081,0.493644,3.766680e-01,0.534668,0.847345,0.270838,5.349500,0.583988,0.476385,1.155429,2.335817e-01,0.804262,0.381554,0.667880,0.737498,0.408957,0.505505,0.538337,1.490681,7.581500,0.487457,4.196949,1.279297,0.124643,0.265294,1.666667,2.471182,0.264115,0.946878,4.579197,0.077555,0.282283,0.000000,0.164745,0.568363,0.625523,1.197594,2.205569,1.245706,0.691051,1.538886e-01,0.000000,2.722201,3.501203e-02,0.108217,0.000000,3.333612,8.793877e-02,0.122008,0.000000,0.722369,1.064654,5.891179,0.000000,0.218100,0.520303,0.000000,0.000000,0.0,0.087289,0.105193,0.240585,0.229502,4.183504,4.320092,1.304506e+00,0.458487,0.446271,0.193528,0.000325,0.080735,0.473258,0.995585,0.140364,0.275002,0.157218,0.761488,2.864203,0.626639,5.682859,0.000000,0.182563,0.165785,2.374763,0.102946,0.320645,0.267974,0.494943,0.608292,1.578062,3.414142e+00,1.219887e+00,0.725611,0.311439,1.017961e+00,0.143230,1.474433,0.365985,0.082788,0.155423,2.025571e-01,0.374756,0.312888,0.000000,3.120291,0.097746,0.000000,0.532411,0.152280,7.557115e-01,1.854353e-01,1.048607,0.365221,0.141842,0.324514,0.265748,0.849412,0.517652,0.145144,0.481634,0.198265,0.513489,0.000325,0.770076,0.447916,0.516145,0.184484,0.849089,0.272550,5.292383,3.389366,1.222309,0.761407,0.386184,0.169521,American Fork,z49
1,ALA,Alta,0.012512,1.334277,0.013142,0.009075,0.001992,0.003969,1.736294,0.000000e+00,0.005027,0.000000,0.009028,0.000000,0.003723,0.000000,0.000000,0.236068,0.000000,0.041776,0.002575,0.000003,0.070859,1.226863e-02,0.001914,0.000000,0.000000e+00,0.008897,0.000000,0.000000,0.000164,0.006163,0.057762,0.004849,0.013505,0.010499,0.000000,0.096431,0.000000,0.000000,0.000000,0.000000,0.007443,0.000000,0.015514,0.002989,0.005949,1.141404e-02,0.008332,0.080251,0.062619,0.012993,0.000000,0.000000,0.047574,0.025482,0.002150,4.556772e-03,0.000514,0.000000,0.004254,0.005942,0.008392,0.000000e+00,0.024827,0.000000,0.002903,0.000000,1.998306e-03

In [66]:
dColCodesWCounty

['AFK_x',
 'ALA_x',
 'ALP_x',
 'BDL_x',
 'BGM_x',
 'BNT_x',
 'BRT_x',
 'CDF_x',
 'CEN_x',
 'CHA_x',
 'CHL_x',
 'CLF_x',
 'CLI_x',
 'CMT_x',
 'COA_x',
 'CWH_x',
 'DAN_x',
 'DRA_x',
 'EAG_x',
 'ELK_x',
 'EMT_x',
 'FAR_x',
 'FCS_x',
 'FFD_x',
 'FRR_x',
 'FTH_x',
 'GLA_x',
 'GOS_x',
 'GRL_x',
 'HAR_x',
 'HDT_x',
 'HEB_x',
 'HER_x',
 'HGH_x',
 'HNF_x',
 'HOL_x',
 'HOO_x',
 'HVL_x',
 'IND_x',
 'INT_x',
 'KAY_x',
 'KMS_x',
 'KMT_x',
 'LAY_x',
 'LEH_x',
 'LIN_x',
 'MAP_x',
 'MID_x',
 'MLC_x',
 'MMT_x',
 'MRG_x',
 'MSL_x',
 'MUR_x',
 'MWY_x',
 'NOG_x',
 'NSL_x',
 'OGD_x',
 'OKL_x',
 'ORM_x',
 'PAY_x',
 'PGR_x',
 'PLN_x',
 'PRK_x',
 'PRY_x',
 'PVO_x',
 'PVW_x',
 'ROY_x',
 'RVD_x',
 'RVT_x',
 'SAN_x',
 'SAQ_x',
 'SAR_x',
 'SFK_x',
 'SJC_x',
 'SLC_x',
 'SLM_x',
 'SOG_x',
 'SPV_x',
 'SSL_x',
 'SUN_x',
 'SWE_x',
 'SYR_x',
 'TAY_x',
 'TOO_x',
 'UIN_x',
 'VIN_x',
 'WAT_x',
 'WBG_x',
 'WDL_x',
 'WEB_x',
 'WHT_x',
 'WHV_x',
 'WIL_x',
 'WJC_x',
 'WPT_x',
 'WVC_x',
 'WXC_x',
 'TTL_x',
 'AFK_y',
 'ALA_y',


In [67]:
df_CountySum            = df_CitySum_wCounty_Join           .groupby(['zCOFIPS'])[dColCodesWCounty].apply(lambda x : x.astype(int  ).sum())
df_CountySum_Percent_SA = df_CitySum_wCounty_Join_Percent_SA.groupby(['zCOFIPS'])[dColCodesWCounty].apply(lambda x : x.astype(float).sum())

df_CountySum            = df_CountySum           .reset_index()
df_CountySum_Percent_SA = df_CountySum_Percent_SA.reset_index()

display(df_CountySum)
display(df_CountySum_Percent_SA)

,zCOFIPS,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,z01,2,0,0,2,2,3,0,0,0,0,1,1,1,0,0,3,1,4,1,0,0,2,0,0,0,0,0,0,1,0,0,1,3,1,0,2,0,0,0,0,1,0,3,5,3,0,1,3,7,3,0,0,2,0,1,3,2,1,4,1,1,0,0,1,22,0,2,1,0,13,3,1,4,2,16,1,0,2,3,1,0,1,8,4,0,0,1,0,1,0,0,2,1,9,0,11,0,2602,5,1,3,1,0,2,0,0,1,0,0,3,0,0,0,3,0,5,0,0,0,5,0,0,1,1,0,0,1,0,0,1,4,1,0,2,0,0,0,0,4,1,1,1,4,4,1,6,11,0,0,0,20,0,1,2,8,1,15,9,2,0,5,0,34,1,2,3,2,13,1,0,4,2,89,0,1,1,15,0,0,1,4,1,0,0,0,0,0,1,0,1,0,10,0,28,2,2290,4,0,21,23,0,0,112,228,1,7,5,2,61,94,2,2,12,19
1,z03,21,0,2,3,3892,46,0,0,10,0,10,129,62,0,2,21,0,28,9,1,1,10,0,0,83,12,4,1,27,117,2,25,11,11,2,28,58,3,0,0,50,4,29,221,32,5,5,29,34,19,20,19,40,5,339,20,976,1,68,10,23,78,14,883,61,164,302,78,28,73,5,14,23,36,169,2,99,35,17,19,12,93,48,87,3,0,74,0,1,4,6,99,242,70,34,104,15,19043,122,1,11,15,5106,109,0,0,58,1,6,501,52,0,2,77,2,223,14,1,3,81,0,0,196,2,3,0,8,79,0,49,11,18,0,40,9,1,0,0,110,5,19,562,114,42,4,99,165,15,14,99,314,4,146,172,2743,0,275,28,40,44,26,546,256,132,260,213,59,295,2,12,66,133,1581,3,171,65,185,20,12,45,143,59,3,3,104,1,0,8,5,123,80,243,23,597,49,26452,5040,5757,832,1924,20,14,878,4416,27,37,119,70,404,1194,34,61,2610,4466
2,z05,74,0,32,21,727,226,1,2,88,7,25,176,101,0,16,70,1,109,74,7,6,99,5,0,46,27,3,6,67,38,4,125,54,54,6,63,46,6,2,1,149,9,74,350,126,33,21,72,127,65,38,15,116,33,119,87,700,9,282,85,101,54,58,157,260,64,268,63,103,230,30,55,100,132,428,22,116,93,52,32,30,106,151,264,11,0,55,1,2,32,13,59,47,248,47,240,57,50124,310,3,27,26,969,208,1,0,112,2,14,328,95,1,3,155,4,406,21,7,0,123,1,0,67,4,2,0,18,96,0,155,34,21,1,74,10,1,0,0,117,4,53,756,322,103,12,214,301,42,26,37,688,14,96,246,1584,1,949,46,94,19,121,60,1063,109,209,301,154,557,12,40,116,263,2901,10,175,75,369,15,19,67,370,149,6,35,67,0,1,38,11,68,12,412,19,860,66,59032,935,1048,1627,2295,38,26,2406,8030,104,132,331,169,1504,3362,175,177,1681,2924
3,z07,13,1,7,1,11,5,0,0,4,0,6,6,5,0,6,8,1,14,19,1,0,3,3,0,2,0,2,1,3,1,0,23,5,15,3,14,2,0,0,1,3,2,14,17,47,4,3,11,27,6,1,1,17,6,4,7,24,2,45,19,14,2,8,2,70,3,11,3,12,25,10,21,32,11,69,10,5,35,6,0,0,5,11,18,0,1,5,0,0,1,2,1,1,44,3,37,2,7702,93,1,16,7,13,10,0,1,3,1,6,12,4,0,0,31,0,141,7,1,0,3,0,0,3,0,2,0,2,0,1,36,12,7,0,18,0,0,0,0,8,4,2,62,68,41,19,55,40,2,0,8,93,2,5,28,110,0,161,44,15,0,12,1,273,0,8,18,25,96,11,9,55,65,594,5,5,44,43,4,1,7,108,23,0,6,0,0,0,2,1,14,1,63,3,180,5,9533,14,15,70,175,1,0,364,1636,25,16,21,26,414,930,31,44,68,188
4,z09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,1,0,214,1,0,0,0,0,1,0,0,0,0,0,2,0,0,0,1,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,1,3,4,0,1,0,3,0,0,0,7,0,0,0,0,0,0,0,0,0,29

,zCOFIPS,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,z01,2.435631e-02,0.000000,0.000000e+00,3.045531e-02,3.052482e-02,1.487948e-02,0.000000,0.000000e+00,1.285563e-08,0.000000e+00,4.511543e-02,0.015007,9.937404e-03,0.000000,0.000000,1.737023e-02,0.182815,2.236539e-02,0.013747,6.973685e-07,0.000000e+00,2.134934e-02,0.000001,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,1.929757e-02,0.000000e+00,0.000000e+00,1.237931e-02,0.029159,1.284380e-02,0.000000e+00,0.013294,1.900237e-07,0.000000,0.000000,0.000000,7.736944e-03,4.565344e-09,2.112797e-02,1.880753e-02,0.013768,0.000000e+00,4.348713e-02,2.909280e-02,2.861959e-02,0.034218,5.370279e-02,0.000000e+00,1.162553e-02,0.000000e+00,1.097815e-02,3.568030e-02,7.552982e-03,1.745201e-01,1.124615e-02,2.052525e-02,6.277082e-03,3.273165e-02,1.565752e-07,3.793684e-02,6.053639e-02,0.000000e+00,1.600321e-02,2.367424e-02,4.193651e-03,0.028678,5.357189e-02,0.014496,3.035146e-02,9.156185e-03,1.906977e-02,0.048697,2.434857e-08,1.808762e-02,3.324700e-02,3.822630e-02,0.000000e+00,1.638763e-02,2.892567e-02,2.951579e-02,0.000000,0.000000,2.214349e-02,0.000000e+00,0.119904,1.377375e-09,0.000000e+00,5.628336e-02,9.832842e-02,0.016904,2.110929e-02,1.877146e-02,1.858143e-08,0.184513,0.031210,1.355896e-01,0.143626,2.271271e-02,5.574802e-07,1.748442e-02,0.000000,0.000000,1.773050e-02,0.529083,0.000000e+00,1.808468e-02,0.000000e+00,0.000000,0.000000e+00,0.014734,0.000000e+00,0.013270,0.000000,0.000000,0.000000,5.465078e-02,0.000000,0.000000,3.244646e-02,0.224215,0.000000,0.000000e+00,0.059313,0.000000e+00,0.000000,0.051510,8.984726e-02,3.302560e-02,0.000000,2.965267e-02,0.000000e+00,0.000000,0.000000,0.0,0.049789,0.086730,2.326664e-02,0.003709,0.015059,0.031957,0.052060,0.032750,0.043643,3.651181e-07,7.974051e-02,5.491192e-02,0.047285,0.000000,3.581662e-02,0.013517,0.015483,3.968254e-01,0.033600,0.176112,0.018718,0.000000e+00,5.136972e-02,0.000000,0.061628,0.049677,3.947851e-02,3.866536e-02,1.960823e-02,0.029204,0.093639,4.283188e-07,0.039302,6.690525e-03,0.035080,2.754775e-06,0.014719,0.023905,0.049941,0.000000,0.000000e+00,2.657483e-02,0.019268,2.591973e-02,0.000000,0.000003,9.918460e-07,0.000000,0.000015,4.610470e-02,0.000000e+00,2.956859e-02,0.000000,0.033542,0.000000e+00,0.043842,4.207912e-02,0.162401,3.708919e-02,4.971687e-07,0.014988,0.021074,5.370279e-02,7.974051e-02,0.020468,0.033669,1.433709e-02,0.057979,2.712220e-02,3.190797e-02,0.024601,0.040846,1.690904e-02,0.053777,0.012008,0.020960
1,z03,2.110920e-01,0.000000,6.711409e-02,6.091061e-02,3.974341e+01,2.727926e-01,0.000000,0.000000e+00,2.281531e-01,0.000000e+00,2.932504e-01,1.050539,7.155901e-01,0.000000,0.521582,1.447559e-01,0.000002,1.476108e-01,0.089353,6.426735e-02,1.224004e-01,1.601405e-01,0.000000,0.000000,3.087367e+00,6

In [68]:
df_CountySum           ['WFCounty'] = "Outside Wasatch Front"
df_CountySum_Percent_SA['WFCounty'] = "Outside Wasatch Front"

df_CountySum           .loc[df_CountySum.zCOFIPS == 'z11', 'WFCounty'] = "Davis County"
df_CountySum           .loc[df_CountySum.zCOFIPS == 'z57', 'WFCounty'] = "Weber County"
df_CountySum           .loc[df_CountySum.zCOFIPS == 'z35', 'WFCounty'] = "Salt Lake County"
df_CountySum           .loc[df_CountySum.zCOFIPS == 'z49', 'WFCounty'] = "Utah County"

df_CountySum_Percent_SA.loc[df_CountySum.zCOFIPS == 'z11', 'WFCounty'] = "Davis County"
df_CountySum_Percent_SA.loc[df_CountySum.zCOFIPS == 'z57', 'WFCounty'] = "Weber County"
df_CountySum_Percent_SA.loc[df_CountySum.zCOFIPS == 'z35', 'WFCounty'] = "Salt Lake County"
df_CountySum_Percent_SA.loc[df_CountySum.zCOFIPS == 'z49', 'WFCounty'] = "Utah County"

display(df_CountySum)
display(df_CountySum_Percent_SA)

,zCOFIPS,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y,WFCounty
0,z01,2,0,0,2,2,3,0,0,0,0,1,1,1,0,0,3,1,4,1,0,0,2,0,0,0,0,0,0,1,0,0,1,3,1,0,2,0,0,0,0,1,0,3,5,3,0,1,3,7,3,0,0,2,0,1,3,2,1,4,1,1,0,0,1,22,0,2,1,0,13,3,1,4,2,16,1,0,2,3,1,0,1,8,4,0,0,1,0,1,0,0,2,1,9,0,11,0,2602,5,1,3,1,0,2,0,0,1,0,0,3,0,0,0,3,0,5,0,0,0,5,0,0,1,1,0,0,1,0,0,1,4,1,0,2,0,0,0,0,4,1,1,1,4,4,1,6,11,0,0,0,20,0,1,2,8,1,15,9,2,0,5,0,34,1,2,3,2,13,1,0,4,2,89,0,1,1,15,0,0,1,4,1,0,0,0,0,0,1,0,1,0,10,0,28,2,2290,4,0,21,23,0,0,112,228,1,7,5,2,61,94,2,2,12,19,Outside Wasatch Front
1,z03,21,0,2,3,3892,46,0,0,10,0,10,129,62,0,2,21,0,28,9,1,1,10,0,0,83,12,4,1,27,117,2,25,11,11,2,28,58,3,0,0,50,4,29,221,32,5,5,29,34,19,20,19,40,5,339,20,976,1,68,10,23,78,14,883,61,164,302,78,28,73,5,14,23,36,169,2,99,35,17,19,12,93,48,87,3,0,74,0,1,4,6,99,242,70,34,104,15,19043,122,1,11,15,5106,109,0,0,58,1,6,501,52,0,2,77,2,223,14,1,3,81,0,0,196,2,3,0,8,79,0,49,11,18,0,40,9,1,0,0,110,5,19,562,114,42,4,99,165,15,14,99,314,4,146,172,2743,0,275,28,40,44,26,546,256,132,260,213,59,295,2,12,66,133,1581,3,171,65,185,20,12,45,143,59,3,3,104,1,0,8,5,123,80,243,23,597,49,26452,5040,5757,832,1924,20,14,878,4416,27,37,119,70,404,1194,34,61,2610,4466,Outside Wasatch Front
2,z05,74,0,32,21,727,226,1,2,88,7,25,176,101,0,16,70,1,109,74,7,6,99,5,0,46,27,3,6,67,38,4,125,54,54,6,63,46,6,2,1,149,9,74,350,126,33,21,72,127,65,38,15,116,33,119,87,700,9,282,85,101,54,58,157,260,64,268,63,103,230,30,55,100,132,428,22,116,93,52,32,30,106,151,264,11,0,55,1,2,32,13,59,47,248,47,240,57,50124,310,3,27,26,969,208,1,0,112,2,14,328,95,1,3,155,4,406,21,7,0,123,1,0,67,4,2,0,18,96,0,155,34,21,1,74,10,1,0,0,117,4,53,756,322,103,12,214,301,42,26,37,688,14,96,246,1584,1,949,46,94,19,121,60,1063,109,209,301,154,557,12,40,116,263,2901,10,175,75,369,15,19,67,370,149,6,35,67,0,1,38,11,68,12,412,19,860,66,59032,935,1048,1627,2295,38,26,2406,8030,104,132,331,169,1504,3362,175,177,1681,2924,Outside Wasatch Front
3,z07,13,1,7,1,11,5,0,0,4,0,6,6,5,0,6,8,1,14,19,1,0,3,3,0,2,0,2,1,3,1,0,23,5,15,3,14,2,0,0,1,3,2,14,17,47,4,3,11,27,6,1,1,17,6,4,7,24,2,45,19,14,2,8,2,70,3,11,3,12,25,10,21,32,11,69,10,5,35,6,0,0,5,11,18,0,1,5,0,0,1,2,1,1,44,3,37,2,7702,93,1,16,7,13,10,0,1,3,1,6,12,4,0,0,31,0,141,7,1,0,3,0,0,3,0,2,0,2,0,1,36,12,7,0,18,0,0,0,0,8,4,2,62,68,41,19,55,40,2,0,8,93,2,5,28,110,0,161,44,15,0,12,1,273,0,8,18,25,96,11,9,55,65,594,5,5,44,43,4,1,7,108,23,0,6,0,0,0,2,1,14,1,63,3,180,5,9533,14,15,70,175,1,0,364,1636,25,16,21,26,414,930,31,44,68,188,Outside Wasatch Front
4,z09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,1,0,214,1,0,0,0,0,1,0,0,0,0,0,2,0,0,0,1,0,2,0,0,0,1,0,0,0,0,0

,zCOFIPS,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y,WFCounty
0,z01,2.435631e-02,0.000000,0.000000e+00,3.045531e-02,3.052482e-02,1.487948e-02,0.000000,0.000000e+00,1.285563e-08,0.000000e+00,4.511543e-02,0.015007,9.937404e-03,0.000000,0.000000,1.737023e-02,0.182815,2.236539e-02,0.013747,6.973685e-07,0.000000e+00,2.134934e-02,0.000001,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,1.929757e-02,0.000000e+00,0.000000e+00,1.237931e-02,0.029159,1.284380e-02,0.000000e+00,0.013294,1.900237e-07,0.000000,0.000000,0.000000,7.736944e-03,4.565344e-09,2.112797e-02,1.880753e-02,0.013768,0.000000e+00,4.348713e-02,2.909280e-02,2.861959e-02,0.034218,5.370279e-02,0.000000e+00,1.162553e-02,0.000000e+00,1.097815e-02,3.568030e-02,7.552982e-03,1.745201e-01,1.124615e-02,2.052525e-02,6.277082e-03,3.273165e-02,1.565752e-07,3.793684e-02,6.053639e-02,0.000000e+00,1.600321e-02,2.367424e-02,4.193651e-03,0.028678,5.357189e-02,0.014496,3.035146e-02,9.156185e-03,1.906977e-02,0.048697,2.434857e-08,1.808762e-02,3.324700e-02,3.822630e-02,0.000000e+00,1.638763e-02,2.892567e-02,2.951579e-02,0.000000,0.000000,2.214349e-02,0.000000e+00,0.119904,1.377375e-09,0.000000e+00,5.628336e-02,9.832842e-02,0.016904,2.110929e-02,1.877146e-02,1.858143e-08,0.184513,0.031210,1.355896e-01,0.143626,2.271271e-02,5.574802e-07,1.748442e-02,0.000000,0.000000,1.773050e-02,0.529083,0.000000e+00,1.808468e-02,0.000000e+00,0.000000,0.000000e+00,0.014734,0.000000e+00,0.013270,0.000000,0.000000,0.000000,5.465078e-02,0.000000,0.000000,3.244646e-02,0.224215,0.000000,0.000000e+00,0.059313,0.000000e+00,0.000000,0.051510,8.984726e-02,3.302560e-02,0.000000,2.965267e-02,0.000000e+00,0.000000,0.000000,0.0,0.049789,0.086730,2.326664e-02,0.003709,0.015059,0.031957,0.052060,0.032750,0.043643,3.651181e-07,7.974051e-02,5.491192e-02,0.047285,0.000000,3.581662e-02,0.013517,0.015483,3.968254e-01,0.033600,0.176112,0.018718,0.000000e+00,5.136972e-02,0.000000,0.061628,0.049677,3.947851e-02,3.866536e-02,1.960823e-02,0.029204,0.093639,4.283188e-07,0.039302,6.690525e-03,0.035080,2.754775e-06,0.014719,0.023905,0.049941,0.000000,0.000000e+00,2.657483e-02,0.019268,2.591973e-02,0.000000,0.000003,9.918460e-07,0.000000,0.000015,4.610470e-02,0.000000e+00,2.956859e-02,0.000000,0.033542,0.000000e+00,0.043842,4.207912e-02,0.162401,3.708919e-02,4.971687e-07,0.014988,0.021074,5.370279e-02,7.974051e-02,0.020468,0.033669,1.433709e-02,0.057979,2.712220e-02,3.190797e-02,0.024601,0.040846,1.690904e-02,0.053777,0.012008,0.020960,Outside Wasatch Front
1,z03,2.110920e-01,0.000000,6.711409e-02,6.091061e-02,3.974341e+01,2.727926e-01,0.000000,0.000000e+00,2.281531e-01,0.000000e+00,2.932504e-01,1.050539,7.155901e-01,0.000000,0.521582,1.447559e-01,0.000002,1.476108e-01,0.089353,6.426735e-02,1.224004e-01,1.601405e-01,0

In [69]:
df_CountySum_wOutWF            = df_CountySum           .groupby(['WFCounty'])[dColCodesWCounty].apply(lambda x : x.astype(int  ).sum())
df_CountySum_wOutWF_Percent_SA = df_CountySum_Percent_SA.groupby(['WFCounty'])[dColCodesWCounty].apply(lambda x : x.astype(float).sum())

df_CountySum_wOutWF            = df_CountySum_wOutWF.reset_index()
df_CountySum_wOutWF_Percent_SA = df_CountySum_wOutWF_Percent_SA.reset_index()

display(df_CountySum_wOutWF)
display(df_CountySum_wOutWF_Percent_SA)

,WFCounty,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,Davis County,184,2,66,114,710,6191,0,2,2536,0,67,5291,3530,10,24,415,8,592,245,22,12,3429,4,0,368,794,9,6,226,385,7,226,400,122,17,475,798,18,0,0,5223,11,615,12961,470,73,60,334,996,489,233,88,733,36,1035,2034,6985,9,747,161,270,354,72,181,686,517,4704,834,648,1260,78,185,329,856,4421,61,1477,261,442,853,1003,5056,930,790,93,1,872,3,2,1027,58,1060,83,1640,1806,2578,1543,111761,648,14,38,75,358,6356,0,1,2826,2,9,7625,993,3,14,1352,0,1538,16,3,0,5490,0,2,383,159,1,0,17,160,0,114,107,42,1,513,56,2,0,0,4950,4,104,13164,732,159,15,948,2156,113,108,203,2978,11,279,4533,11288,7,1285,54,139,66,181,18,1175,172,1880,1930,238,2261,5,39,107,1118,32680,10,1501,103,3015,467,206,1968,1187,232,29,17,639,0,0,893,7,576,6,1300,504,6105,2172,148169,1048,408,54691,53893,233,108,19349,59709,181,231,1054,268,4825,5552,322,143,20574,20260
1,Outside Wasatch Front,423,6,203,109,4827,664,3,13,259,102,165,551,330,6,348,652,269,763,569,60,39,283,234,2,163,59,41,27,1681,190,168,4714,390,350,164,624,167,12,48,17,432,531,564,1214,1178,169,161,459,1365,444,560,41,808,1067,619,267,2459,316,1574,512,502,173,2498,1090,1720,300,894,206,645,1662,343,473,882,833,3622,185,342,768,471,90,94,403,933,5205,19,1,212,63,33,74,94,248,308,1743,150,1809,144,268291,1779,34,547,203,6291,745,3,5,385,110,54,1294,225,3,390,1180,76,2534,122,33,6,524,34,5,337,20,15,3,1239,219,2,3903,179,136,21,532,30,3,6,0,517,811,240,2669,1814,812,120,1307,1999,208,771,198,3807,757,320,1382,6802,127,4100,596,634,73,7334,614,5304,287,740,904,725,3247,95,199,1007,1818,22268,93,726,1053,2482,67,51,231,1642,5049,18,82,276,16,6,142,24,342,98,2399,64,6424,336,323074,6269,7051,5675,9719,560,771,19548,55543,4235,8854,6935,6330,11605,20177,6629,4978,6543,12059
2,Salt Lake County,2967,101,1424,4972,1474,10943,45,95,3621,52,1289,3708,2504,304,186,14512,96,16694,6600,239,593,4452,77,20,445,834,137,69,2345,434,39,1638,13811,2919,55,12471,629,38,20,7,5286,174,16013,10041,11660,945,772,14678,26667,12297,370,102,22048,544,1492,5028,7309,92,9048,1930,3470,420,790,393,6672,636,3602,742,19020,40685,1098,6074,3896,26816,80401,797,1534,3347,9931,571,764,3401,26683,8458,111,10,770,34,162,1599,2359,902,155,50576,1175,55062,2707,686752,4441,1083,351,2600,279,2264,31,7,1041,9,95,1087,157,28,54,14366,13,24541,275,16,74,1175,3,26,111,47,5,0,167,59,3,385,2999,455,1,7444,27,4,1,0,576,66,3165,2906,8817,1718,67,12940,19196,2049,32,88,30499,79,98,5453,3998,26,7336,321,1325,19,2823,18,6710,61,349,763,6964,34283,38,590,655,21287,173264,71,470,944,23720,57,42,248,14893,1182,10,80,74,4,9,458,183,125,3,26860,51,47034,1118,563927,2144,307,58993,19029,370,32,472316,475304,1548,3079,10969,1433,69145,37423,2643,570,20496,7281
3,Utah County,8008,5,2286,885,410,741,2,1

,WFCounty,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,Davis County,1.907908,2.339181,2.125272,2.329807,8.048580,31.289918,2.631423,0.931798,35.323259,0.000000,2.255775,40.579262,36.222368,3.218804,5.867034,2.825497,3.656279,2.983480,2.068817,2.185007,2.570670,37.809098,2.921437,0.000000,14.023440,39.936230,1.739107,2.685853,5.190960,14.776176,4.129776,3.466073,2.723460,2.209026,8.379787,3.615962,24.062180,12.186676,1.666667,2.000000,41.353851,2.403327,3.644549,41.025615,1.879166,2.030327,1.826476,2.309978,3.415199,3.757101,15.843958,13.499242,3.154338,2.728392,12.427826,25.354617,17.893614,2.617817,1.848889,2.144875,2.008601,13.781577,2.838868,8.649609,1.922055,14.317731,25.718304,21.781476,3.044570,2.781768,1.767831,1.667000,1.972930,2.881030,4.801215,1.923458,19.981294,1.929306,4.230619,35.741701,35.245532,42.420612,3.265349,5.112194,20.775374,1.111111,21.081189,4.762052,0.839334,34.961105,2.805283,21.558122,11.504925,2.946361,40.278752,4.190682,31.696038,7.929241,3.415080,2.372932,2.585354,3.111590,5.080198,56.686033,3.333333,1.923153,52.390459,1.587745,3.313588,46.755160,43.805796,6.578947,3.460141,7.273026,1.091106,4.421472,1.847408,1.612903,2.962436,61.425944,1.666403,1.980198,15.479348,53.808503,3.084164,0.000000,2.253792,14.976685,0.000000,3.279431,3.795954,2.774076,6.521739,6.157459,14.990656,14.864542,7.692308,0.0,63.268084,1.474495,3.862051,49.314430,2.581061,1.949350,1.717861,5.878349,8.313480,5.796408,13.159780,14.938852,6.997810,3.082014,13.039124,31.505628,19.648477,4.365111,2.973503,1.504897,2.030841,12.311308,2.362994,4.194400,2.233923,12.221948,26.375238,26.422625,3.284208,4.989526,1.123597,2.629795,1.375576,4.157848,12.651123,1.061170,23.951266,1.482072,9.235695,47.572349,44.338771,55.514705,6.324391,4.250715,21.929746,3.692879,22.325484,1.851852,1.613054,46.286349,2.590793,19.931185,8.197909,3.991336,57.102407,9.110004,48.343958,10.509442,8.426847,5.057151,37.355977,49.514537,15.843958,13.159780,3.496863,8.761053,3.383602,2.384415,5.130645,3.892620,1.928606,2.420020,3.271715,3.145867,19.130967,20.360721
1,Outside Wasatch Front,4.075976,5.847940,5.749596,2.131924,49.826971,3.730488,5.263272,5.279542,4.174174,49.211725,4.805308,4.704921,3.885769,2.145922,49.543225,4.186266,55.391981,3.820142,4.529620,5.463851,6.242759,3.533614,59.998831,3.797367,6.601333,3.725900,7.454032,7.025668,33.616136,7.338175,59.880934,59.580604,2.711948,5.445984,53.630835,4.666369,5.444031,6.453490,54.999265,57.999783,3.953695,59.873660,3.454571,4.122207,4.478042,4.100766,4.479740,3.078119,4.674615,3.634147,32.115175,5.939819,3.456745,49.075084,7.421610,3.735009,6.470706,60.557683,3.813481,6.106372,3.704123,6.776623,66.141757,42.829981,4.624558,8.425026,5.238664,5.753054,3.069875,3.664755,7.178848,4.022754,4.992987,2

In [70]:
# bu map unit
# block group
df_CountySum_wOutWF_Percent_MU = df_CountySum_wOutWF.copy()

for x in dColCodesWCounty_x:   
    #display(x)
    df_CountySum_wOutWF_Percent_MU[x] = df_CountySum_wOutWF_Percent_MU[x] / df_CountySum_wOutWF_Percent_MU['TTL_x'] * 100

for y in dColCodesWCounty_y:   
    #display(y)
    df_CountySum_wOutWF_Percent_MU[y] = df_CountySum_wOutWF_Percent_MU[y] / df_CountySum_wOutWF_Percent_MU['TTL_y'] * 100

df_CountySum_wOutWF_Percent_MU

,WFCounty,AFK_x,ALA_x,ALP_x,BDL_x,BGM_x,BNT_x,BRT_x,CDF_x,CEN_x,CHA_x,CHL_x,CLF_x,CLI_x,CMT_x,COA_x,CWH_x,DAN_x,DRA_x,EAG_x,ELK_x,EMT_x,FAR_x,FCS_x,FFD_x,FRR_x,FTH_x,GLA_x,GOS_x,GRL_x,HAR_x,HDT_x,HEB_x,HER_x,HGH_x,HNF_x,HOL_x,HOO_x,HVL_x,IND_x,INT_x,KAY_x,KMS_x,KMT_x,LAY_x,LEH_x,LIN_x,MAP_x,MID_x,MLC_x,MMT_x,MRG_x,MSL_x,MUR_x,MWY_x,NOG_x,NSL_x,OGD_x,OKL_x,ORM_x,PAY_x,PGR_x,PLN_x,PRK_x,PRY_x,PVO_x,PVW_x,ROY_x,RVD_x,RVT_x,SAN_x,SAQ_x,SAR_x,SFK_x,SJC_x,SLC_x,SLM_x,SOG_x,SPV_x,SSL_x,SUN_x,SWE_x,SYR_x,TAY_x,TOO_x,UIN_x,VIN_x,WAT_x,WBG_x,WDL_x,WEB_x,WHT_x,WHV_x,WIL_x,WJC_x,WPT_x,WVC_x,WXC_x,TTL_x,AFK_y,ALA_y,ALP_y,BDL_y,BGM_y,BNT_y,BRT_y,CDF_y,CEN_y,CHA_y,CHL_y,CLF_y,CLI_y,CMT_y,COA_y,CWH_y,DAN_y,DRA_y,EAG_y,ELK_y,EMT_y,FAR_y,FCS_y,FFD_y,FRR_y,FTH_y,GLA_y,GOS_y,GRL_y,HAR_y,HDT_y,HEB_y,HER_y,HGH_y,HNF_y,HOL_y,HOO_y,HVL_y,IND_y,INT_y,KAY_y,KMS_y,KMT_y,LAY_y,LEH_y,LIN_y,MAP_y,MID_y,MLC_y,MMT_y,MRG_y,MSL_y,MUR_y,MWY_y,NOG_y,NSL_y,OGD_y,OKL_y,ORM_y,PAY_y,PGR_y,PLN_y,PRK_y,PRY_y,PVO_y,PVW_y,ROY_y,RVD_y,RVT_y,SAN_y,SAQ_y,SAR_y,SFK_y,SJC_y,SLC_y,SLM_y,SOG_y,SPV_y,SSL_y,SUN_y,SWE_y,SYR_y,TAY_y,TOO_y,UIN_y,VIN_y,WAT_y,WBG_y,WDL_y,WEB_y,WHT_y,WHV_y,WIL_y,WJC_y,WPT_y,WVC_y,WXC_y,TTL_y,z03_x,z03_y,z11_x,z11_y,z29_x,z29_y,z35_x,z35_y,z43_x,z43_y,z45_x,z45_y,z49_x,z49_y,z51_x,z51_y,z57_x,z57_y
0,Davis County,0.164637,0.001790,0.059055,0.102003,0.635284,5.539499,0.000000,0.001790,2.269128,0.000000,0.059949,4.734210,3.158526,0.008948,0.021474,0.371328,0.007158,0.529702,0.219218,0.019685,0.010737,3.068154,0.003579,0.000000,0.329274,0.710445,0.008053,0.005369,0.202217,0.344485,0.006263,0.202217,0.357907,0.109162,0.015211,0.425014,0.714024,0.016106,0.000000,0.000000,4.673365,0.009842,0.550281,11.597069,0.420540,0.065318,0.053686,0.298852,0.891187,0.437541,0.208481,0.078739,0.655864,0.032212,0.926083,1.819955,6.249944,0.008053,0.668391,0.144057,0.241587,0.316747,0.064423,0.161953,0.613810,0.462594,4.208982,0.746235,0.579809,1.127406,0.069792,0.165532,0.294378,0.765920,3.955763,0.054581,1.321570,0.233534,0.395487,0.763236,0.897451,4.523939,0.832133,0.706866,0.083213,0.000895,0.780236,0.002684,0.001790,0.918925,0.051896,0.948453,0.074266,1.467417,1.615948,2.306708,1.380625,100.0,0.437338,0.009449,0.025646,0.050618,0.241616,4.289696,0.000000,0.000675,1.907282,0.001350,0.006074,5.146151,0.670181,0.002025,0.009449,0.912472,0.000000,1.038004,0.010798,0.002025,0.000000,3.705228,0.000000,0.001350,0.258489,0.107310,0.000675,0.000000,0.011473,0.107985,0.000000,0.076939,0.072215,0.028346,0.000675,0.346226,0.037795,0.001350,0.000000,0.0,3.340780,0.002700,0.070190,8.884450,0.494030,0.107310,0.010124,0.639810,1.455095,0.076264,0.072890,0.137006,2.009867,0.007424,0.188298,3.059344,7.618328,0.004724,0.867253,0.036445,0.093812,0.044544,0.122158,0.012148,0.793013,0.116084,1.268821,1.302567,0.160627,1.525960,0.003375,0.026321,0.072215,0.754544,22.055896,0.006749,1.013032,0.069515,2.034839,0.315181,0.139030,1.328213,0.801112,0.156578,0.019572,0.011473,0.431264,0.000000,0.000000,0.602690,0.004724,0.388745,0.004049,0.877377,0.340152,4.120295,1.465894,100.0,0.937715,0.275361,48.935675,36.372656,0.208481,0.072890,17.312837,40.297903,0.161953,0.155903,0.943084,0.180875,4.317248,3.747073,0.288115,0.096511,18.408926,13.673575
1,Outside Wasatch Front,0.157665,0.002236,0.075664,0.040628,1.799166,0.247492,0.001118,0.004845,0.096537,0.038018,0.061500,0.205374,0.123001,0.002236,0.129710,0.243020,0.100264,0.284393,0.212083,0.022364,0.014536,0.105482,0.087219,0.000745,0.060755,0.021991,0.015282,0.010064,0.626558,0.070819,0.062619,1.757047,0.145365,0.130455,0.061128,0.232583,0.062246,0.004473,0.017891,0.006336,0.161019,0.197919,0.210220,0.452494,0.439075,0.062991,0.060009,0.171083,0.508776,0.165492,0.208729,0.015282,0.301166,0.397702,0.230720,0.099519,0.916542,0.117783,0.586676,0.190838,0.187110,0.064482,0.931079,0.406275,0.641095,0.111819,0.333220,0.076782,0.240411,0.619477,0.127846,0.176301,0.328748,0.310484,1.350027,0.068955,0.127474,0.286256,0.175556,0.033546,0.035037,0.150210,0.3477

In [71]:
df_CountySum_wOutWF           .to_json(os.path.join(dirResults, "countydata_number.json"    ),orient='records')
df_CountySum_wOutWF_Percent_SA.to_json(os.path.join(dirResults, "countydata_percent_sa.json"),orient='records')
df_CountySum_wOutWF_Percent_MU.to_json(os.path.join(dirResults, "countydata_percent_mu.json"),orient='records')